# Weird Python

This notebook is meant as a fun little adventure for those who have been using Python for a bit and want to see some of the rabbitholes it has to offer.

In fact, almost any programming language offers these rabbit holes. They are, by their very nature, inherent to what we call "programming".

Or, as we might also call it, magic.

## Prelude in ƒ -- code as data

We know how to define functions:

In [2]:
def inc(x):
    """Increment a number"""
    return x + 1

print(inc(1))
print(inc(7))

2
8


But did you know that a function argument, or any value for that matter, can also be a function?
This way, in our code, we can actually *talk* about code (not just run it).

In [3]:
# (look up first-class functions if you want to dive deeper)

def do_something(x, the_doing_func):
    """Take a value x and a function, and apply the function to the value x."""
    return the_doing_func(x)

do_something(3, inc)

4

`inc` is just any old name (like a variable) that points to something.
In this case, it points to a function. `def` is really just a funny syntax for assigning a bunch of code to a name.

Note the lack of parenthesis after `inc`.
Adding a parenthesis (and maybe some arguments) after `inc`, like `inc(3)`, *calls* or runs the function.
That's why we won't add them when we *talk about* functions (assign them, pass them around, etc.)

In [4]:
# Aside:
# You might have heard about lambda functions. Those are just function definitions (bunches of code) for when you don't need a name.

# So this does the same, but without the step of calling the function `inc`
do_something(3, lambda a: a + 1)

# Unfortunately, lambdas are kinda hobbled in Python, so for non-trivial functions, you need to use `def`.

4

So code is itself really just another form of data. This makes it easy to sepearate *what* happens from *how* it happens. Which, in turn, makes it easy to implement code that is more dynamic than one might be used to. We can even use the programming language's own data structures like `dict` and `list` to manage our code snippets:

In [5]:
funcs = {
    "add": lambda a_tup: a_tup[0] + a_tup[1],
    "sq": lambda a: a * a,
}
transformations = ["add", "sq", "sq"]

def transform(value):
    for func_name in transformations:
        func = funcs[func_name]
        value = func(value)
    return value

transform((2,3))

625

And, while it doesn't matter for the sake of the argument, of course we can skip the func_name strings and use symbols instead:

In [355]:
def add(a_tup):
    return a_tup[0] + a_tup[1]

def sq(a):
    return a * a

transformations = [add, sq, sq]

def transform(value):
    for func in transformations:
        value = func(value)
    return value

transform((2,3))

625

This makes it easy to change the handling of the data flow centrally, e.g. for auditing or for debugging:

In [358]:
def transform(value):
    for func in transformations:
        print(f"{value} --{func.__name__}--> ", end="")
        value = func(value)
        print(value)
    return value

transform((2,3))

(2, 3) --add--> 5
5 --sq--> 25
25 --sq--> 625


625

If we want to keep our imperative style and don't like to specify pipelines like we did above, we can also introduce generic logic decentrally.

One way is to wrap the intermediary result to make it possible to react to things like invalid intermediary results, but without cluttering our code up with if statements.

In [870]:
def success(val):
    return val()[0] if callable(val) else lambda: (True, val)  # checks val (if callable) or constructs a value
    
def failure(err):
    return not err()[0] if callable(err) else lambda: (False, err)  # checks err (if callable) or constructs an error

def value_of(val):
    return val()[1] if success(val) else None

def error_of(val):
    return val()[1] if failure(val) else None

val = success  # shortcut to create a value (fresh values are so far successful)

def guarded(func):
    def wrapper(*args):
        any_failure = [v for v in args if failure(v)]
        if any_failure:
            return any_failure[0]
        raw_args = [value(v) for v in args]
        try:
            raw_result = func(*raw_args)
        except Exception as e:
            return failure(f"Error in {func.__name__}(): {e}")
        return success(raw_result)
    return wrapper

# Now that we have defined some generic functions for wrapping, let's use them:

@guarded
def div(a, b):
    print(f"dividing {a} / {b}...")
    return a / b

@guarded
def add(a, b):
    print(f"adding {a} + {b}...")
    return a + b

@guarded
def sq(a):
    print(f"squaring {a}...")
    return a * a

transformations = [div, sq, sq]

def transform(*args):
    data = [success(arg) for arg in args]
    for func in transformations:
        func_args = data if isinstance(data, (tuple, list)) else (data, )
        data = func(*func_args)
    return data

# Examples:

print("First calculation:")
result1 = transform(2, 3)
print(f"{value(result1)=}, {error(result1)=}")  # All steps ran successfully
print("\nSecond calculation:")
result2 = transform(2, 0)
print(f"{value(result2)=}, {error(result2)=}") # No unnecessary steps will run, and we keep orchestration free from case-by-case error handling

# It might look like we could achieve the same with exceptions, and that's certainly possible,
# particularly if we have a higher-order pipeline function such as transform here which can host generic handling logic.
# But this also protects any style of coding, even if it's imperative style with/without assignments, etc.:

print("\nThird calculation:")
added = add(val(3), val(5))
divided = div(added, val(2))
added2 = add(added, divided)
result3 = sq(added2)
print(f"{value_of(result3)=}")
print(f"{error_of(result3)=}")

print("\nFourth calculation:")
result4 = add(sq(val(3)),
              div(val(3), val(2))
             )
print(f"{value_of(result4)=}")
print(f"{error_of(result4)=}")

print(f"...or with div/0 somewhere in the middle: {error_of(add(sq(val(3)), div(val(3), val(0))))=}")

# We will look at ways to make the syntax more intuitive at another time.

First calculation:
dividing 2 / 3...
squaring 0.6666666666666666...
squaring 0.4444444444444444...
value(result1)=0.19753086419753085, error(result1)=None

Second calculation:
dividing 2 / 0...
value(result2)=None, error(result2)='Error in div(): division by zero'

Third calculation:
adding 3 + 5...
dividing 8 / 2...
adding 8 + 4.0...
squaring 12.0...
value_of(result3)=144.0
error_of(result3)=None

Fourth calculation:
squaring 3...
dividing 3 / 2...
adding 9 + 1.5...
value_of(result4)=10.5
error_of(result4)=None
squaring 3...
dividing 3 / 0...
...or with div/0 somewhere in the middle: error_of(add(sq(val(3)), div(val(3), val(0))))='Error in div(): division by zero'


Let's revisit the `inc` example. But this time, we make our function return another function that does the incrementing. Just for fun, we're also introducing a function that will return a function that delivers a specific value for us:

In [703]:
def val(x):
    def inner():
        return x
    return inner

def inc(x_func):
    def inner():
        return x_func() + 1
    return inner  # we could also return a lambda function -- same thing

val3 = val(3)
inc3 = inc(val3)
print(inc3)
print(inc3())

<function inc.<locals>.inner at 0x1105a9820>
4


That way, we've introduced another *indirection*: Not only can our code talk about incrementing any number, it can now also talk about incrementing a specific number -- without actually having to do the operation (with in some cases might be rather expensive and worth delaying until (if at all) we actually need the value)

In [8]:
inc4 = inc(inc3)

So what's going on here? Well, remember that calls to `inc` and `val` didn't *do* anything, but just return functions that will do something? We're just moving along that same chain of *promising-to-do-something-in-the-future-just-not-now* some more. :)

Only when we decide to call the last thing that we have by tacking parenthesis to it, everything will happen at once:

In [9]:
inc4
inc4()

5

To reflect briefly, the examples you've seen above can be viewed as two different ways to implement data pipelines: The first defines the pipeline in an external data structure (`transformations`). The second is chaining the pipeline steps themselves to a data-delivery function made by `val` and to each other.

Maybe the separation between functions (that are kind of abstracted for certain types of things to do) and the functions made by them (that are concrete that can actually be used) reminds you of classes in object-oriented programming. If that is so, congrats for finding new links between programming concepts. Just try to not equate the concepts. OOP is a bundle of concepts that has been pre-selected for you to work with. Here, we're exploring the fundamentals that might have informed some choices made for different programming paradigms, be it imperative, functional, or other.

In [877]:
# We can take the idea of code as data further by not even using "ordinary" numbers or arithmetics in our code,
# but let the code *be* the arithmetics.

# In the following, f is kind of the increment function from earlier, only not defined explicitly.
# Numbers here are just the notion of doing "something" (f) one, two, ... some number of times.
zero = lambda f: (lambda x: x)  # The parentheses are just for clarity here
one = lambda f: (lambda x: f(x))
two = lambda f: (lambda x: f(f(x)))
three = lambda f: (lambda x: f(f(f(x))))

# We can test it by anything where we can see how often something has happend such as:
print(f"{three(lambda x: x + '.')('')=}")  # '...'
print(f"{two(lambda x: x + 1)(0)=}")  # 2

# We are nesting the lambda parameters so that we get partial evaluation by default.

# Let's try some arithmetic:
two_times_three = lambda g: two(three(g))  # 6
# λg.two(three(g)) = λg.two(λx.gggx) = λg.(λf.λx.ffx(λx.gggx)) = λg.(λx.(λx.gggx)(λx.gggx)x) = λg.(λx.(λx.gggx)gggx) = λg.λx.ggggggx
# Application: λg.λx.ggggggx(inc)(0) = λx.inc(inc(inc(inc(inc(inc(x))))))(0) = inc(inc(inc(inc(inc(inc(0))))))
print(f"{two_times_three(lambda x: print(f'{x}->{x+1}') or x + 1)(0)=}")

# For addition, we somehow have to (temporarily) get rid of the outermost λf, fill the λx with the other number, and re-afffix the λf.λx.
two_plus_three = lambda f: lambda x: two(f)(three(f)(x))
print(f"{two_plus_three(lambda x: print(f'{x}->{x+1}') or x + 1)(0)=}")

# Or more generally:
mult = lambda a: lambda b: lambda g: a(b(g))
add = lambda a: lambda b: lambda f: lambda x: a(f)(b(f)(x))
power = lambda base: lambda exp: exp(base)  # stole that one from wikipedia :)

# A helper for examining the result
show = lambda f: f(lambda x: x + 1)(0)

print(f"{show(add(two)(three))=}")
print(f"{show(add(one)(mult(three)(three)))=}")
print(f"{show(power(two)(three))=}")

# General incrementer (also called successor). It is equivalent to `add` applied to `one`
inc = lambda b: lambda f: lambda x: f(b(f)(x))  # = (lambda a: lambda b: lambda f: lambda x: a(f)(b(f)(x)))(one)
print(f"{show(inc(three))=}")
    
# How to decrement (positive) numbers?
# We need to get rid of exactly one f in the f(...f(x)) chain.
# A first idea maybe would be to consume it and replace it by an identity function.
# But a simple substitution means that we'd replace *every* f with identity, making the result 0.
# two = lambda f: (lambda x: f(f(x)))
# two = lambda f: (lambda x: f(f( lambda x: lambda f: 0 )))
four = inc(three)

# I think we have to pass an f that can build another sequence of nested lambdas.
"""
Let's try something
I: two = λf.λx.f(f(x))
II: λf.λg.g(f)
I applied to II: λx.(λf.λg.g(f))((λf.λg.g(f))(x))
= λx.(λf.λg.g(f))(λg.g(x))
= λx.(λg.g(λg.g(x)))
= λx.λg.g(λg.g(x)) (III)
IV: λx.x
Apply III to IV: (λx.λg.g(λg.g(x)))(λx.x)
= λg.g(λg.g(λx.x))
= λg.g(λx.x)
= λx.x ... darn lol...

Okay, it seems that Church & his students themselves did not solve the PRED thing, but someone else did later.
Not saying it wasn't out of choice, but I get why this part is hard. :D
"""

three(lambda x: x + '.')('')='...'
two(lambda x: x + 1)(0)=2
0->1
1->2
2->3
3->4
4->5
5->6
two_times_three(lambda x: print(f'{x}->{x+1}') or x + 1)(0)=6
0->1
1->2
2->3
3->4
4->5
two_plus_three(lambda x: print(f'{x}->{x+1}') or x + 1)(0)=5
show(add(two)(three))=5
show(add(one)(mult(three)(three)))=10
show(power(two)(three))=8
show(inc(three))=4
<function <lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x106440b80>
<function <lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x1129d7040>
<function <lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x1129d7670>
<function <lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x1129d70d0>


1

In [699]:
# Boolean values and logic are a bit more straightforward to implement.

# A true value returns the first parameter, a false value returns the second parameter
true = lambda l: lambda r: l
false = lambda l: lambda r: r

print(f"{true(1)(0)=}")
print(f"{false(1)(0)=}")

negate = lambda a: lambda l: lambda r: a(r)(l)  # not

# Another helper for testing
bshow = lambda f: f(1)(0)

print(f"{bshow(negate(true))=}")
print(f"{bshow(negate(false))=}")

# While it works to write the l/r parameters out explicitly and apply them in our new definitions,
# and this is in fact the only way available to us in proper lambda calculus,
# let's reuse our previous definitions to make things shorter and clearer:
negate = lambda a: a(false)(true)
print(f"{bshow(negate(true))=}")
print(f"{bshow(negate(false))=}")

# Boolean conjunction (AND):
conjunct = lambda a: lambda b: lambda l: lambda r: a(b(l)(r))(r)  # Could still be shortened significantly, but hopefully somewhat clear

# Clearer when using earlier definitions:
conjunct = lambda a: lambda b: a(b)(false)  # much clearer, but could even be shortened further to a(b)(a)

print(f"{bshow(conjunct(true)(true))=}")
print(f"{bshow(conjunct(false)(true))=}")
print(f"{bshow(conjunct(true)(false))=}")
print(f"{bshow(conjunct(false)(false))=}")

# Boolean OR (disjunction)
disjunct = lambda a: lambda b: lambda l: lambda r: a(l)(b(l)(r))  # or
disjunct = lambda a: lambda b: a(true)(b)  # Could be shortened to a(a)(b)

print(f"{bshow(disjunct(true)(true))=}")
print(f"{bshow(disjunct(false)(true))=}")
print(f"{bshow(disjunct(true)(false))=}")
print(f"{bshow(disjunct(false)(false))=}")

# XOR for fun - not really needed, could be composed of or, not, and
# xor is the same as boolean equality, by the way
xor = beq = lambda a: lambda b: lambda l: lambda r: a(b(l)(r))(b(r)(l))
# Shortening that one is left as an exercise :)

print(f"{bshow(xor(true)(true))=}")
print(f"{bshow(xor(false)(true))=}")
print(f"{bshow(xor(true)(false))=}")
print(f"{bshow(xor(false)(false))=}")


true(1)(0)=1
false(1)(0)=0
bshow(negate(true))=0
bshow(negate(false))=1
bshow(negate(true))=0
bshow(negate(false))=1
bshow(conjunct(true)(true))=1
bshow(conjunct(false)(true))=0
bshow(conjunct(true)(false))=0
bshow(conjunct(false)(false))=0
bshow(disjunct(true)(true))=1
bshow(disjunct(false)(true))=1
bshow(disjunct(true)(false))=1
bshow(disjunct(false)(false))=0
bshow(xor(true)(true))=1
bshow(xor(false)(true))=0
bshow(xor(true)(false))=0
bshow(xor(false)(false))=1


By the way -- I'm trying to avoid Python constructs like for loops, lists, list comprehensions, etc. in the next segment. I'm not doing that because I wouldn't like them, but to show that they are, in fact, optional. :)

Let's blur the distinction between data and code even more (with apologies to H. Abelson and G.J. Sussman):

## Structured incantations -- code *is* data

In [42]:
def pair(a, b):
    def get(which):
        if which == 0:
            return a
        elif which == 1:
            return b
    return get

one_two = pair(1, 2)
one_two

<function __main__.pair.<locals>.get(which)>

In [43]:
def first(a_pair):
    return a_pair(0)

def second(a_pair):
    return a_pair(1)

print(first(one_two))
print(second(one_two))

1
2


What we did here is have a pair-creation function, that actually does not store any values in any data structure, but just *returns a function* that is able to retrieve those two values.

Big deal, we've got two values. That's hardly a data structure.

It isn't?

In [52]:
my_list = pair(1, pair(2, pair(3, pair(4, None))))  # None signifies the empty list = end of the list

def print_list(l):
    if l is None:
        return ""
    return f"{first(l)}\n{print_list(second(l))}"

print(print_list(my_list))

# A little helper
def lst(*elems):
    return None if not elems else pair(elems[0], make_list(*elems[1:]))

print(print_list(lst(1, 2, 3, 4)))

1
2
3
4

1
2
3
4



Actually, it is! You might have heard of linked lists. This is one of them. `first` gets the head of the list, and `second` gets the remainder or tail.

[ Note that the function `print_list` is calling itself to get from one item to the next (recursion), instead of using Python's iteration mechanisms like for loops. Recursion and iteration are internally two sides of the same coin. Unfortunately, again, Python is somewhat not clean in having a fixed maximum recursion depth. We find ways to circumvent that for specific "tail recursive" cases later on. ]

We can also create trees:

In [76]:
my_bin_tree = pair(pair(pair(0, 1), 2), pair(pair(3, 4), pair(pair(5, pair(6, None)), 7)))

def print_bin_tree(t, indent=0):
    if not callable(t):
        return " "*indent + str(t) + "\n"
    else:
        return " "*indent + "*\n" + print_bin_tree(first(t), indent+1) + print_bin_tree(second(t), indent+1)

print(print_bin_tree(my_bin_tree))  # The asterisks are unnamed nodes (only leaf nodes have values)

*
 *
  *
   0
   1
  2
 *
  *
   3
   4
  *
   *
    5
    *
     6
     None
   7



Here's a tree made of lists (which are, as we remember, None-terminated):

In [83]:
another_tree = lst(0, lst(1, 2, lst("a", "b", "c")), "x", lst(42), lst(9, 8, 7))

def print_tree(t, indent=0):
    if t is None:
        return ""
    head, tail = first(t), second(t)
    if not callable(head):
        return " "*indent + str(head) + "\n" + print_tree(tail, indent)
    else:
        return " "*indent + "*\n" + print_tree(head, indent+1) + print_tree(tail, indent)

print(print_tree(another_tree))
# We still have unnamed nodes. Can you think of a lst and/or pair structure that allows for naming all nodes?

0
*
 1
 2
 *
  a
  b
  c
x
*
 42
*
 9
 8
 7



And here's something similar to a dictionary (an association list), and make our print_list more flexible while we're at it:

In [84]:
my_assoc_list = pair(pair("bob", 42), pair(pair("susan", 51), pair(pair("harry", 23), pair(pair("lisa", 27), None))))

def print_list(l, fmt=lambda x: f"{x}\n"):
    if l is None:
        return ""
    return f"{fmt(first(l))}{print_list(second(l), fmt=fmt)}"

def print_assoc(l, fmt=lambda assoc_pair: f"pair({first(assoc_pair)}, {second(assoc_pair)})\n"):
    return print_list(l, fmt=fmt)

print(print_assoc(my_assoc_list))

pair(bob, 42)
pair(susan, 51)
pair(harry, 23)
pair(lisa, 27)



Now, I get it, defining a data structure like this looks clumsy and tedious, but let me remind you that we never used any pre-existing data structure -- no arrays, no lists, no dicts, no tuples. Just functions.

But just to demonstrate that this could be made perfectly usable, let me provide some convenience functionality:

In [85]:
def get(assoc_list, key):
    if assoc_list is None:
        return None
    elif key == first(first(assoc_list)):
        return second(first(assoc_list))
    else:
        return get(second(assoc_list), key)

print("Getting some dict values more conveniently:")
print( get(my_assoc_list, "querty") )
print( get(my_assoc_list, "harry") )

def make_list(*values):  # readable version of lst ;)
    if not values:
        return None
    return pair(values[0], make_list(*values[1:]))

print("\nCreating a list more conveniently:")
l2 = make_list(1,2,3,4)
print(print_list(l2))

print("Creating an assoc list more conveniently\n(in this case to work like an array, for which we could also make a convenience function of course, which would be a nice exercise):")
al2 = make_list(pair(0, "Aaron"), pair(1, "Berta"), pair(2, "Caesar"))
print(print_assoc(al2))
print(get(al2, 1))

Getting some dict values more conveniently:
None
23

Creating a list more conveniently:
1
2
3
4

Creating an assoc list more conveniently
(in this case to work like an array, for which we could also make a convenience function of course, which would be a nice exercise):
pair(0, Aaron)
pair(1, Berta)
pair(2, Caesar)

Berta


A more complex tree structure can also be made by nesting assoc-lists, of course, but I will leave this as an exercise for the reader. :)

My main points with all the data structure examples above are:
1. Data structures aren't "baked-in". Anyone can make any data structure. From scratch.
2. If there isn't a convenient way to handle data, make one.
3. You *can* create something from nothing. Remember that we still don't have a single variable inside our data structure. It's all "just procedures".

## If you want to talk about something, you'll first have to invent a word for it

Now we'll be turning the whole thing upside down a bit. What if all you have is data (e.g. a string), and you want to make the computer actually do something according to it?

Okay, your very own programming language coming right up.

It's almost lunch time here, so let's keep it simple.

In [16]:
program = """
set(x, add(3, 4))
set(y, add(5, 6))
print(mult(x, y))
"""

I've chosen a syntax that's simple to parse: Every instruction consists of a function name, followed by parentheses which contain the arguments. Commas separate the arguments.

You can see that also `=` and `+` are written that way. It's just a superficial difference and our choice makes it easier for us right now. Support for more wide-spread syntactic structures can always be added later as syntactic sugar.

Let's first think through what this code actually does. We do this by replacing some evaluated instruction by its result, step by step, until we have an end result:

Start:
```
set(x, add(3, 4))
set(y, add(5, 6))
print(mult(x, y))
```
We cannot do "set", "mult" or "print" rightaway because they depend on other results.
So we first find the innermost expressions and replace them by what they represent.

3, 4, 5 and 6 represent themselves, so there's no change for them. Let's move further out.

Replacing "add(3, 4)" with its result 7 (its return value):
```
set(x, 7)
set(y, add(5, 6))
print(mult(x, y))
```

We can now do the first "set" and replace it with its result. "set" does not have a return value, so the result is NOTHING:
```
NOTHING
set(y, add(5, 6))
print(mult(x, y))
```

NOTHING stands in its own expression, as an orphan. It is not used anywhere in a surrounding function, so it's ignored. The choice of NOTHING instead of Python's None arbitrary. I chose it to show that we're free to make our own choices.

So did "set" have an effect? It did, but it's not visible. It bound the value 7 to a name called "x", stored somewhere else, invisible from us. You might find that this increases the burden of the programmer having to remember things that are invisible, and you would be correct. Things that happen invisibly from the code replacement view are called "side effects". There are whole programming paradigms invented to make this problem go away. But I'm trying to keep it more conventional for Pythonistas here.

Replacing "add(5, 6)" with its result 11:
```
NOTHING
set(y, 11)
print(mult(x, y))
```

Replacing "set(y, 11)" with its result NOTHING:
```
NOTHING
NOTHING
print(mult(x, y))
```

Replacing "x" with its stored value 7 and "y" with its stored value 11:
```
NOTHING
NOTHING
print(mult(7, 11))
```

Replacing "mult(7, 11)" with its result 77:
```
NOTHING
NOTHING
print(77)
```

Replacing "print(77)" with its result NOTHING (printing "77" is another side effect):
```
NOTHING
NOTHING
NOTHING
```

Now, the complete program has been evaluated. It actually also has a return value. It is the last instruction NOTHING. But we don't do anything with it, so it really does not matter.

Phew. Walking through this complete program that way has been kind of tedious. But it's useful to see the individual steps that happen. And to get rid of the tediousness, we'll make the computer do it. :)

Our first step will be to separate the string into the parts of the code that matter to us. Here' I'll use Python data structures again, so Python users can grasp the concepts more easily. You are welcome to modify this code to use data structures like the ones we created from thin air in the previous section. :)

In [17]:
program = """
set(x, add(3, 4))
set(y, add(5, 6))
print(mult(x, y))
"""

def parse(expr):
    curr_expression = []  # the first item will be the name of the operation, any remaining items will be the arguments
    super_expressions = []  # a stack to remember which expression belongs inside which other expression
    curr_token = ""
    for char in expr:
        # print(f"{curr_expression=}")
        if char == "(":  # got a subexpression
            sub_expression = [curr_token]  # got operator name
            curr_token = ""
            curr_expression.append(sub_expression)  # sub_expression at current position in current expression's argument list
            super_expressions.append(curr_expression)  # remember where we descended from
            curr_expression = sub_expression  # continue with the subexpression
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)  # got last argument & end of expression
                curr_token = ""
            curr_expression = super_expressions.pop()  # back to last remembered super expression
        elif char in [" ", "\n", "\r"]:
            pass
        elif char == ",":
            if curr_token:
                curr_expression.append(curr_token)  # got argument
                curr_token = ""
        else:
            curr_token += char
    if curr_token:
        curr_expression.append(curr_token)
    return curr_expression

parse(program)

[['set', 'x', ['add', '3', '4']],
 ['set', 'y', ['add', '5', '6']],
 ['print', ['mult', 'x', 'y']]]

I have deliberately chosen a non-trivial syntax and an iterative parsing approach (which uses `super_expressions` as a stack) which could be implemented in pretty much any language, even if it does not support recursion.

Had I used a LISP/SCHEME-like syntax like `(command arg1 arg2)`, it would be a bit simpler. Also, using recursion instead of iteration (which would Python's call stack as stack) would make it shorter. Here's a recursive parser for a lisp-like language. Why not try to adapt it to our syntax as an exercise? ;) Also, feel invited to try to make it more concise and clear using regexes or other fancy stuff.

In [18]:
lisp_program = """
(set x (add 3 4))
(set y (add 5 6))
(print (mult x y))
"""  # setting global variables with "set" is not really very LISP-like, but let's ignore that...

def lisp_parse_recursive(expr, pos=None):
    curr_expression = []
    curr_token = ""
    if pos is None:
        pos = [0, 0]  # pos[0] = curr pos and pos[1] = end pos
    while pos[0] < len(expr):
        char = expr[pos[0]]
        pos[0] += 1
        if char == "(":
            curr_expression.append( lisp_parse_recursive(expr, pos=pos) )
            pos[0] = pos[1]  # continue where the subexpression ended
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)
                curr_token = ""
            pos[1] = pos[0]  # communicate where this subexpression ended
            return curr_expression
        elif char in [" ", "\n", "\r"]:
            if curr_token:
                curr_expression.append(curr_token)
                curr_token = ""
        else:
            curr_token += char
    return curr_expression


lisp_parse_recursive(lisp_program)

[['set', 'x', ['add', '3', '4']],
 ['set', 'y', ['add', '5', '6']],
 ['print', ['mult', 'x', 'y']]]

Back to our original program -- let's run it!

In [19]:
program = """
set(x, add(3, 4))
set(y, add(5, 6))
print(mult(x, y))
"""

global_names = {  # Only contains our operations initially
    "add": lambda a: int(a[0]) + int(a[1]),
    "mult": lambda a: int(a[0]) * int(a[1]),
    "print": lambda a: print(', '.join([str(x) for x in a])),
    "set": lambda a: global_names.update({a[0]: a[-1]}),
}

def evaluate(expr, level=0):
    if not isinstance(expr, list):
        return global_names.get(expr, expr)  # If it's not a name, regard expr as a literal (get()'s fallback value)
    else:
        evaluated_expr = []
        for item in expr:
            evaluated_expr.append(evaluate(item, level + 1))
        if level > 0:
            op = evaluated_expr[0]
            args = evaluated_expr[1:]
            result = op(args)
        else:
            result = evaluated_expr[-1]  # Last result in imperative-style list of commands counts as return value
        return result if result is not None else "NOTHING"

evaluate(parse(program))  # It prints 77 and returns NOTHING because the print command does not have a return value

77


'NOTHING'

That was simple. Even surprisingly so, maybe.

In recursion, always look out for two conditions: the stop condition, where we return a concrete result, and the recursion condition, wherein we still have to decompose our problem and are doing a recursive call with the parts. Look for the return statements. Here, the following branches lead to a returned (partial) result: If expr is an atomic value (that might or might not be a name), we simply return it (no further recursion). If expr is non-atomic, we evaluate all its parts (which is recursive and eventually makes sure that all parts are atomic before we continue). Then, we actually apply the command to its arguments and return the result. This happens with top-level expressions, but also with any subexpressions, sub-subexpressions, etc., reducing them all to a flat list of atoms.

You can of course make this stricter, like requiring all lookups to actually succeed, separating strings from other types, throwing error messages, explicit returns, etc. But the less strict it is, the less clutter there is, and the more fun to be had. ;)

# Side quest: decorators

Defining commands with only lambda functions is somewhat limiting (many things that we want to do are not allowed in Python lambdas). So let's create a nicer way to create our commands, which is, coincidentally, also a nicely weird concept: decorators.

In [20]:
global_names = {}
def command(types=(), name=None):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        return typed_func
    return decorate

@command(types=(int, int))
def add(a, b):
    return a + b

@command(types=(int, int))
def mult(a, b):
    return a * b

@command(types=(str,), name="print")  # don't want to overwrite any built-in symbols
def print_cmd(a):
    print(a)

@command(types=(str,), name="set")
def set_cmd(key, val):
    global_names[key] = val

ast = parse(program)
evaluate(ast)

77


'NOTHING'

It is a good idea for this exercise to only implement our language's primitives that way. That way, we test the capabilities and usability of our language interpreter while we extend it with more commands. As always, don't optimize too early, it just clutters up everything.

### Our very own stack. Overflows included.

One reason that the `evaluate` function was so nice and quick, was that we are surfing on Python's own call stack to handle our interpreter's call stack for us. So let's roll our own, and learn a bunch along the way:

In [21]:
def evaluate(expr):
    stack=[]
    curr_expr = expr
    curr_atoms = []
    while True:
        if len(curr_atoms) == len(curr_expr): # Fully evaluated current expression.
            if curr_expr is expr: # We're done
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            # We can apply the operator now
            op = curr_atoms[0]
            args = curr_atoms[1:]
            result = op(args)
            result = result if result is not None else "NOTHING"
            curr_expr, curr_atoms = stack.pop()  # Restore previous environment
            curr_atoms.append(result)
        else:
            next_item = curr_expr[len(curr_atoms)]
            if isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_expr, curr_atoms)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                item_val = global_names.get(next_item, next_item)  # If it's not a name, regard expr as a literal (get()'s fallback value)
                curr_atoms.append(item_val)

# Also add another little convenience function
def run(prog):
    return evaluate(parse(prog))

run(program)

77


'NOTHING'

It's not that much more complex actually. I added a few more comments for clarity, but it's still only about twice as long as our recursive version. 

Remember that we don't do any recursion here, but iteratio. But still, we do have a somewhat similar structure of conditions: The first conditions says that we're done simplifying/evaluating the current expression to atoms, and can apply the command to the arguments and get a result. The other condition is again the one that decomposes the expression further and descends the syntax tree.

But the main difference here is that we descend and ascend the tree without recursion, so we have to keep track of the `curr_exp` we are working on, and of any results so far (`curr_atoms`). We note down our state of affairs in the current expression before we deal with a subexpression (the part dealing with `stack_frame`). And, vice-versa, once we're done with a subexpression, we get the state from the superexpression (`stack.pop()`), add the subexpression's result atom to it, and continue with its remaining unevaluated items.

So, as we descend down into subexpressions, the `stack` grows, and as we ascend back, the `stack` shrinks. When we're done, it's empty.

To understand this code better, add a few prints of the stack and current variables.

Concering the section title, I lied. Actually we can't get stack overflows because we don't enforce a maximum stack depth (for reference, my local Python interpreter has a stack depth of 3000), so it would run possibly much longer, until Python runs out of memory.

Which it can't, because we can't do recursion, because our language does not have user-defined functions.

Let's fix that.

### User-defined functions

Adding the capability of user-defined functions just means adding another built-in command:

In [22]:
@command(types=(), name="lambda")
def lambda_cmd(arg_names, func_body):
    print(f"{arg_names=}")
    print(f"{func_body=}")
    def inner(*args):
        for arg_name, arg in zip(arg_names, args):
            global_names[arg_name] = arg
        return evaluate(func_body)
    return inner

ast = parse("lambda((a, b), add(a, b))")
evaluate(ast)

TypeError: 'str' object is not callable

Oh no! `evaluate` is trying to execute our lambda function immediately. In fact, it even tries to execute the function's parameter declaration.

So it appears we have a couple things to fix.

`lambda` introduces a new class of commands whose evaluation is delayed. There are actually more commands like this: the first parameter of `set` also should have delayed evaluation (if you try `set(x, 7)` and `set(x, 3)` one after another, the second is executed as `set(7, 3)` and the number 7 will never be the same again. :D

In [ ]:
run("set(x, 7) set(x, 3) 7")  # whoops :)

Our `evaluate` has to skip evaluation of the arguments of non-postposed commands. The delayed commands are then responsible for evaluating their own arguments. This is just a matter of adding a global register for them, as well an `is_delayed` variable and a corresponding `if` statement in `evaluate`:

In [ ]:
delayed_names = []

def evaluate(expr):
    stack = []
    curr_expr = expr
    curr_atoms = []
    is_delayed = False
    preceding_lambda = None
    while True:
        if len(curr_atoms) == len(curr_expr): # Fully evaluated current expression.
            if curr_expr is expr: # We're done
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            if curr_atoms[0] or preceding_lambda:  # If we have an operator or, if it's '', we are following a lambda. Hacky but our syntax choices makes this necessary to support immediate lambda application
                # We can apply the operator now
                op = global_names[curr_atoms[0]] if curr_atoms[0] else preceding_lambda
                args = curr_atoms[1:]
                result = op(args)
                result = result if result is not None else "NOTHING"
            if not curr_atoms[0] and preceding_lambda is None:
                # No operator to apply something to: leave it as a list (sans empty operator '')
                result = curr_atoms[1:]
            curr_expr, curr_atoms = stack.pop()  # Restore previous environment
            curr_atoms.append(result)
            is_delayed = False  # No need to make is_delayed part of the stack frame as we're not descending from delayed expressions anyway
            if callable(result):
                preceding_lambda = result
            else:
                preceding_lambda = None
        else:
            next_item = curr_expr[len(curr_atoms)]
            is_first = not curr_atoms
            is_delayed = is_delayed or (is_first and next_item in delayed_names)
            if is_delayed:
                curr_atoms.append(next_item)
            elif isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_expr, curr_atoms)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                if not is_first:
                    next_item = global_names.get(next_item, next_item)  # If it's not a name, regard expr as a literal (get()'s fallback value)
                curr_atoms.append(next_item)

Let's also tweak our `command` decorator and redefine `set` and `lambda`:

In [ ]:
def command(types=(), name=None, delayed=False):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        if delayed:
            delayed_names.append(name)
        return typed_func
    return decorate


@command(types=(str,), name="set", delayed=True)
def set_cmd(key, val):
    evaluated = evaluate([val])
    global_names[key] = evaluate([val])


# @command(types=(), name="lambda", delayed=True)
# def lambda_cmd(arg_names, *func_body):
#     arg_names = arg_names[1:]  # The first item is an 'empty' command from the parser. Remove it
#     def inner(args):
#         for i, arg_name in enumerate(arg_names):
#             if arg_name.startswith("*"):  # Support for variable-length arguments
#                 global_names[arg_name] = args[i:]
#                 break
#             else:
#                 global_names[arg_name] = args[i]
#         result = evaluate(list(func_body))
#         return result
#     return inner

Trying it out:

In [ ]:
run("set(l, lambda((a, b), add(add(a, b), b)))")
run("l(2, 3)")

It works!

We can even immediately apply anonymous lambdas due to a small hack with `preceding_lambda` in `evaluate`:

In [ ]:
run("lambda((a, b), mult(add(a, b), b))(1,2)")

So what does our `global_names` environment look like now?

In [ ]:
global_names

You might not expect `a` and `b` to show up there. This way, parameters will overwrite other stuff (not that we cared much so far about names being overwritten, but still).

Fortunately for us, it's pretty straightforward to make environments like `global_names`, but local to the functions that use them. And we can build the lookup so that it can also access the surrounding environments. It pretty much looks like another stack:

In [ ]:
def evaluate(expr, env=None):
    if env is None:
        env = (global_names, None)  # Tuple of (current env, link to env above)
    stack = []
    curr_expr = expr
    ...

Hmm, wait... I think we need to change more than just adding a linked list of envs. We would have to redefine the lambda function and the set function to have another parameter for the current env.

Also, having the lambda function as its own Python function defeats the purpose of having a stack-based `evaluate`: With recursion of lambdas going on, `evaluate` would just call `lambda` which would call `evaluate` and so on, which would side-step our DIY stack and 100% rely on Python's call stack.

We need to stay inside `evaluate` for our recursion, that is, extend its expressions dynamically with the lambda body whenever a lambda is called (plus stuff like binding the params and managing the envs). To this end, lambda will not be any old command defined as a Python function, but we'll try to completely stay inside `evaluate` when dealing with it (helper functions that don't call `evaluate` are allowed).

Also, we need to capture the environment of the definition of the lambda, and then bind the parameters (in new sub-environment) before executing a lambda.

In [ ]:
# Version 1: Add env, "non-stack" if, but stuck on tail call optimization:

STACK_SIZE = 100
global_names = {}

def coerce(a_string):
    try:
        return int(a_string)
    except ValueError:
        pass
    try:
        return float(a_string)
    except ValueError:
        return str(a_string)

def coerce_bool(a_string):
    num_bool = bool(coerce(a_string))
    if str(a_string).lower() == "false" or not num_bool:
        return "FALSE"
    elif str(a_string).lower() == "true" or num_bool:
        return "TRUE"


def command(types=(), name=None):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        return typed_func
    return decorate

@command(types=(int, int))
def add(a, b):
    return a + b

@command(types=(int, int))
def sub(a, b):
    return a - b

@command(types=(int, int))
def mult(a, b):
    return a * b

@command(types=(str,), name="print")  # don't want to overwrite any built-in symbols
def print_cmd(a):
    print(a)

@command(types=())
def eq(a, b):
    return "TRUE" if coerce(a) == coerce(b) else "FALSE"

def parse(expr):
    curr_expression = []  # the first item will be the name of the operation, any remaining items will be the arguments
    super_expressions = []  # a stack to remember which expression belongs inside which other expression
    curr_token = ""
    touching_parens = False
    for char in expr:
        # print(f"{curr_expression=}")
        if char == "(":  # got a subexpression
            if touching_parens:
                # Previous complex expression is meant to be applied to the following argument list (which has no operator)
                sub_expression = [curr_expression.pop()]  # Move complex expression to beginning of subexpression
            else:
                sub_expression = [curr_token]  # got operator name
            curr_token = ""
            curr_expression.append(sub_expression)  # sub_expression at current position in current expression's argument list
            super_expressions.append(curr_expression)  # remember where we descended from
            curr_expression = sub_expression  # continue with the subexpression
            touching_parens = False
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)  # got last argument & end of expression
                curr_token = ""
            curr_expression = super_expressions.pop()  # back to last remembered super expression
            touching_parens = True
        elif char in [" ", "\n", "\r"]:
            touching_parens = False
            pass
        elif char == ",":
            if curr_token:
                curr_expression.append(curr_token)  # got argument
                curr_token = ""
            touching_parens = False
        else:
            curr_token += char
            touching_parens = False
    if curr_token:
        curr_expression.append(curr_token)
    return curr_expression

def lookup(name, env):
    value = None
    while env is not None:
        try:
            value = env[0][name]
            return value
        except KeyError:
            env = env[1]
    return name  # If it's not a name, regard expr as a literal (get()'s fallback value)

def bind(args, params, parent_env):
    sub_env = {}
    for i, param in enumerate(params):
        if param.startswith("*"):  # Support for variable-length arguments
            sub_env[param] = args[i:]
            break
        else:
            sub_env[param] = args[i]
    return (sub_env, parent_env)


def evaluate(expr, env=None):
    if env is None:
        env = (global_names, None)  # Linked list of tuple(curr env, next env tuple)
    stack = []
    curr_expr = expr
    curr_atoms = []
    curr_op = None
    in_closure = None  # TODO: Tail call optimization
    while True:
        if len(stack) > STACK_SIZE:
            raise RuntimeError(f"Max stack size of {STACK_SIZE} exceeded.")
        # print(f"{curr_expr=}")
        # print(f"{curr_atoms=}")
        if len(curr_atoms) == len(curr_expr): # Fully evaluated current expression.
            if curr_expr is expr and curr_op != "if":  # The 'if' part is a stupid hack because we mix expr lists and exprs
                # We're done
                # print(f"{env=}")
                # print("done", len(stack))
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            # We can apply the operator now
            op = curr_atoms[0]
            args = curr_atoms[1:]
            if isinstance(op, list) and op[0] == "closure":
                # print(f"Got closure, evaluating it")
                lambda_params = op[1][0][1:]  # split off preceding empty op ''
                lambda_body = op[1][1:]
                lambda_env = op[2]
                # print(f"{lambda_params=}, {lambda_body=}, {lambda_env=}")
                sub_env = bind(args, lambda_params, lambda_env)
                # Save current environment
                stack_frame = ("closure", curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = [''] + lambda_body  # Treat the result as a list, not a command
                curr_atoms = []
                env = sub_env
                # print(stack_frame)
                curr_op = None
            else:
                # print(op, args)
                if op == 'lambda':
                    # print(f"Making closure from lambda function")
                    # Create a closure of args (params, *function body), capturing the current env
                    # Note that this is assigned as result and will thus not be evaluated further unless called at a later point.
                    result = ["closure", args, env]
                elif op == 'set':
                    # print(f"Setting {args[0]}={args[-1]}")
                    env[0][args[0]] = args[-1]
                    result = "NOTHING"
                elif curr_op == 'if':
                    # print(f"if args: {args}")
                    # result = args[1] if coerce_bool(args[0]) == "TRUE" else (args[2] if len(args) > 2 else "NOTHING")
                    # Whatever the last value will be the branch that fired:
                    result = args[1] if len(args) > 0 else "NOTHING"
                elif op:  # any primitive operator that is not the empty string
                    result = op(args)
                    result = result if result is not None else "NOTHING"
                else:
                    # No operator to apply something to (it's just a list). Leave it as a list (sans empty operator name '')
                    result = curr_atoms[1:]
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Restore previous environment
                # print(f"Returning from {curr_op}")
                if curr_op == "closure":
                    curr_op, curr_expr, curr_atoms, env = stack.pop()  # Get rid of the closure layer -- its result is uninterpretable otherwise
                    # print(f"{curr_atoms=}")
                    # print(f"Picking last result {result[-1]} from result {result}")
                    result = result[-1]
                curr_atoms.append(result)
        else:
            next_item = curr_expr[len(curr_atoms)]
            is_first = not curr_atoms
            if is_first:
                curr_op = next_item
            # print(f"Evaluating {next_item} ({len(curr_atoms)} of {len(curr_expr)-1}) {curr_op=}, {env[0]=}")
            # print(curr_op, len(curr_atoms), curr_atoms)
            if curr_op == 'lambda':
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of all args
            elif curr_op == 'set' and len(curr_atoms) == 1:
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of first arg
            elif curr_op == 'if' and ((len(curr_atoms) == 2 and coerce_bool(curr_atoms[1]) != "TRUE") or
                                      (len(curr_atoms) == 3 and coerce_bool(curr_atoms[1]) == "TRUE")):
                    # print(f"{curr_op} with cond={bool(curr_atoms[1])} at {len(curr_atoms)}, so not evaluating branch")
                    # Completely skip the non-firing branch - we don't need it at all, not even unevaluated
                    # curr_atoms.append(next_item)  # delay evaluation of first arg      
                    curr_expr.pop()  # Re-write the incoming expression
                    pass
            elif isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_op, curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                next_item = lookup(next_item, env)
                # print(f"{next_item=}")
                curr_atoms.append(next_item)
        print(f"{curr_op=}, {len(stack)=}")

def run(prog):
    ast = parse(prog)
    # print(f"{ast=}")
    return evaluate(ast)


# run("set(l, lambda((a,b) add(a, b) add(b, a)))")
# run("l(3,add(5, 5))")
# run("lambda((a,b) add(a, b) add(b, a))(1,2)")
# run("""
# set(incby, lambda((a), lambda((b) add(b, a))))
# set(plustwo, incby(2))
# plustwo(4)
# """)
# run("""
# set(rec, lambda((x), print(x) rec(add(x, 1))))
# rec(0)
# """)
# run("add(1,3)")
# run("set(a,1), set(b,1) if(eq(a,b), 1, 2)")
run("if(FALSE, 1, 2)")
# run("""
# set(fact, lambda((x) if(eq(x, 1), 1, mult(x, fact(sub(x, 1))))))
# fact(5)
# """)  # 120
# run("""
# set(fact,
#     lambda((x)
#            set(iter, lambda((n, acc)
#                             if(eq(n, 1),
#                                acc,
#                                iter(sub(n, 1), mult(n, acc)))
#                            )
#               ) 
#             iter(x, 1)
#           )
#    )
# fact(6)
# """)  # 120

In [102]:
# Version 2: Moved closure to evaluation part, with limited success

STACK_SIZE = 100
global_names = {}

def coerce(a_string):
    try:
        return int(a_string)
    except ValueError:
        pass
    try:
        return float(a_string)
    except ValueError:
        return str(a_string)

def coerce_bool(a_string):
    num_bool = bool(coerce(a_string))
    if str(a_string).lower() == "false" or not num_bool:
        return "FALSE"
    elif str(a_string).lower() == "true" or num_bool:
        return "TRUE"


def command(types=(), name=None):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        return typed_func
    return decorate

@command(types=(int, int))
def add(a, b):
    return a + b

@command(types=(int, int))
def sub(a, b):
    return a - b

@command(types=(int, int))
def mult(a, b):
    return a * b

@command(types=(int, int))
def div(a, b):
    return a / b

@command(types=(str,), name="print")  # don't want to overwrite any built-in symbols
def print_cmd(a):
    print(a)

@command(types=())
def eq(a, b):
    return "TRUE" if coerce(a) == coerce(b) else "FALSE"

@command(types=(), name="not")
def not_cmd(x):
    return "TRUE" if coerce(x) == "FALSE" else "FALSE"

def parse(expr):
    curr_expression = []  # the first item will be the name of the operation, any remaining items will be the arguments
    super_expressions = []  # a stack to remember which expression belongs inside which other expression
    curr_token = ""
    touching_parens = False
    for char in expr:
        # print(f"{curr_expression=}")
        if char == "(":  # got a subexpression
            if touching_parens:
                # Previous complex expression is meant to be applied to the following argument list (which has no operator)
                sub_expression = [curr_expression.pop()]  # Move complex expression to beginning of subexpression
            else:
                sub_expression = [curr_token]  # got operator name
            curr_token = ""
            curr_expression.append(sub_expression)  # sub_expression at current position in current expression's argument list
            super_expressions.append(curr_expression)  # remember where we descended from
            curr_expression = sub_expression  # continue with the subexpression
            touching_parens = False
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)  # got last argument & end of expression
                curr_token = ""
            curr_expression = super_expressions.pop()  # back to last remembered super expression
            touching_parens = True
        elif char in [" ", "\n", "\r"]:
            touching_parens = False
            pass
        elif char == ",":
            if curr_token:
                curr_expression.append(curr_token)  # got argument
                curr_token = ""
            touching_parens = False
        else:
            curr_token += char
            touching_parens = False
    if curr_token:
        curr_expression.append(curr_token)
    return curr_expression

def lookup(name, env):
    value = None
    while env is not None:
        try:
            value = env[0][name]
            return value
        except KeyError:
            env = env[1]
    return name  # If it's not a name, regard expr as a literal (get()'s fallback value)

def bind(args, params, parent_env, update=False):
    sub_env = {}
    for i, param in enumerate(params):
        if param.startswith("*"):  # Support for variable-length arguments
            sub_env[param] = args[i:]
            break
        else:
            sub_env[param] = args[i]
    if update:
        parent_env.update(sub_env)  # Overwrite any names that are present (desired for iteration/tail call optimization)
        return parent_env
    else:
        return (sub_env, parent_env)  # Create a new sub-environment

import time

def evaluate(expr, env=None):
    if env is None:
        env = (global_names, None)  # Linked list of tuple(curr env, next env tuple)
    stack = []
    curr_expr = expr
    curr_atoms = []
    curr_op = "closure"
    in_closure = None  # TODO: Tail call optimization
    while True:
        time.sleep(0.05)
        if len(stack) > STACK_SIZE:
            raise RuntimeError(f"Max stack size of {STACK_SIZE} exceeded.")
        print(f"{curr_expr=}")
        print(f"{curr_atoms=}")
        if len(curr_atoms) == len(curr_expr): # Fully evaluated current expression.
            if curr_expr is expr:
                # We're done
                # print(f"{env=}")
                # print("done", len(stack))
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            # We can apply the operator now
            op = curr_atoms[0]
            args = curr_atoms[1:]
            print(f"Applying {op=}, {args=}")
            if isinstance(op, list) and op[0] == "closure":
                print(f"Got closure, evaluating it")
                lambda_params = curr_atoms[0][1][0][1:]  # [1:] to split off preceding empty op ''
                lambda_body = curr_atoms[0][1][1:]  # Body can contain a series of imperative-style expressions
                lambda_captured_env = curr_atoms[0][2]
                lambda_args = curr_atoms[1:]
                print(f"{lambda_params=}, {lambda_body=}, {lambda_captured_env=}, {lambda_args=}")
                # A new stack frame has already been created when entering the closure apply and it has saved the previous env.
                # So we don't need another stack frame here and can overwrite the env variable:
                env = bind(lambda_args, lambda_params, lambda_captured_env)
                # Rewrite the current expression to have it applied:
                curr_expr = [''] + lambda_body  # Treat the result as a list of expressions, not a single expression/command
                curr_atoms = []
                curr_op = "closure"
                # # print(f"Got closure, evaluating it")
                # lambda_params = op[1][0][1:]  # split off preceding empty op ''
                # lambda_body = op[1][1:]
                # lambda_env = op[2]
                # # print(f"{lambda_params=}, {lambda_body=}, {lambda_env=}")
                # sub_env = bind(args, lambda_params, lambda_env)
                # # Save current environment
                # stack_frame = ("closure", curr_expr, curr_atoms, env)
                # stack.append(stack_frame)
                # # Activate a new environment for this subexpression
                # curr_expr = [''] + lambda_body  # Treat the result as a list, not a command
                # curr_atoms = []
                # env = sub_env
                # # print(stack_frame)
                # curr_op = None
            else:
                # print(op, args)
                if op == 'lambda':
                    # print(f"Making closure from lambda function")
                    # Create a closure of args (params, *function body), capturing the current env
                    # Note that this is assigned as result and will thus not be evaluated further unless called at a later point.
                    result = ["closure", args, env]
                elif op == 'set':
                    # print(f"Setting {args[0]}={args[-1]}")
                    env[0][args[0]] = args[-1]
                    result = "NOTHING"
                elif op:  # any primitive operator that is not the empty string
                    result = op(args)
                    result = result if result is not None else "NOTHING"
                else:
                    # No operator to apply something to (it's just a list). Leave it as a list (sans empty operator name '')
                    result = curr_atoms[1:]
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Restore previous environment
                # print(f"Returning from {curr_op}")
                # if curr_op == "closure":
                #     curr_op, curr_expr, curr_atoms, env = stack.pop()  # Get rid of the closure layer -- its result is uninterpretable otherwise
                #     # print(f"{curr_atoms=}")
                #     # print(f"Picking last result {result[-1]} from result {result}")
                #     result = result[-1]
                print(f"{curr_op=}, {result=}")
                if curr_op == "closure" and isinstance(result, list):
                    result = result[-1]
                curr_atoms.append(result)
        else:
            next_item = curr_expr[len(curr_atoms)]
            print(f"{curr_op=}, {next_item=}, {len(stack)=}")
            is_first = not curr_atoms
            if is_first and not isinstance(next_item, list):
                curr_op = next_item
            # print(f"Evaluating {next_item} ({len(curr_atoms)} of {len(curr_expr)-1}) {curr_op=}, {env[0]=}")
            # print(curr_op, len(curr_atoms), curr_atoms)
            if curr_op == 'lambda':
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of all args
            elif curr_op == 'set' and len(curr_atoms) == 1:
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of first arg
            elif curr_op == 'if' and len(curr_atoms) >= 2:
                print(f"{curr_op} with cond={bool(curr_atoms[1])} at {len(curr_atoms)}: choosing branch")
                # Rewrite the current expression and replace it with the firing expression
                if coerce_bool(curr_atoms[1]) == "TRUE":
                    new_expr = curr_expr[2]
                else:
                    new_expr = curr_expr[3] if len(curr_expr) >= 4 else "UNKNOWN"
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Throw away stack frame made for if
                curr_expr[len(curr_atoms)] = new_expr  # Make the result directly part of the previous expr level
                print(f"After if: {curr_expr=}, {curr_atoms=}")
            # elif len(curr_atoms) == len(curr_expr)-1   aoseuthoesuteohuo ush !!!!!! and isinstance(curr_atoms[0], list) and curr_atoms[0] and curr_atoms[0][0] == "closure":
            #     print(f"Got closure, evaluating it")
            #     lambda_params = curr_atoms[0][1][0][1:]  # [1:] to split off preceding empty op ''
            #     lambda_body = curr_atoms[0][1][1:]  # Body can contain a series of imperative-style expressions
            #     lambda_captured_env = curr_atoms[0][2]
            #     lambda_args = curr_atoms[1:]
            #     print(f"{lambda_params=}, {lambda_body=}, {lambda_captured_env=}, {lambda_args=}")
            #     # A new stack frame has already been created when entering the closure application and it has saved the previous env.
            #     # So we don't need another stack frame here and can overwrite the env variable:
            #     env = bind(lambda_args, lambda_params, lambda_captured_env)
            #     # Rewrite the current expression to have it applied:
            #     curr_expr = [''] + lambda_body  # Treat the result as a list of expressions, not a single expression/command
            #     curr_atoms = []
            elif isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_op, curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                next_item = lookup(next_item, env)
                # print(f"{next_item=}")
                curr_atoms.append(next_item)

def run(prog):
    ast = parse(prog)
    # print(f"{ast=}")
    return evaluate(ast)

# assert run("add(1,3)") == 4
# run("set(l, lambda((a,b) add(a, b) add(b, a)))")
# assert run("l(3,add(5, 5))") == 13
# assert run("set(l, lambda((a,b) add(a, b) add(b, a))) l(1,2)") == 3
assert run("lambda((a,b) add(a, b) add(b, a))(1,2)") == 3
# assert run("""
# set(incby, lambda((a), lambda((b) add(b, a))))
# set(plustwo, incby(2))
# plustwo(4)
# """) == 6
# run("""
# set(rec, lambda((x), print(x) rec(add(x, 1))))
# rec(0)
# """)  # = Infinite recursion
# assert coerce(run("set(a,1), set(b,1) 3")) == 3
# assert run("if(TRUE, 1, 2)") == '1'
# assert run("if(FALSE, 1, 2)") == '2'

# assert run("set(a,3), set(b,0) if(eq(a,0), div(a, b), div(b, a))") == 0.0

# assert run("""
# set(fact, lambda((x) if(eq(x, 1), 1, mult(x, fact(sub(x, 1))))))
# fact(5)
# """) == 120
# assert run("""
# set(fact,
#     lambda((x)
#            set(iter, lambda((n, acc)
#                             if(eq(n, 1),
#                                acc,
#                                iter(sub(n, 1), mult(n, acc)))
#                            )
#               ) 
#             iter(x, 1)
#           )
#    )
# fact(5)
# """) == 120

curr_expr=[[['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']], '1', '2']]
curr_atoms=[]
curr_op='closure', next_item=[['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']], '1', '2'], len(stack)=0
curr_expr=[['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']], '1', '2']
curr_atoms=[]
curr_op='closure', next_item=['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']], len(stack)=1
curr_expr=['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']]
curr_atoms=[]
curr_op='closure', next_item='lambda', len(stack)=2
curr_expr=['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']]
curr_atoms=['lambda']
curr_op='lambda', next_item=['', 'a', 'b'], len(stack)=2
curr_expr=['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']]
curr_atoms=['lambda', ['', 'a', 'b']]
curr_op='lambda', next_item=['add', 'a', 'b'], len(stack)=2
curr_expr=['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']]
curr_atoms=['lambda', [''

TypeError: 'tuple' object is not callable

In [55]:
# Version 3 (in progress): Find a way to only put stuff on the stack for real applies (not if, not set, not closures)
# The goal is to automatically have iterative tail calls if the last expression is a call to the same closure, but without
# any additional "optimization" (ideally)

STACK_SIZE = 100
global_names = {}

def coerce(a_string):
    try:
        return int(a_string)
    except ValueError:
        pass
    try:
        return float(a_string)
    except ValueError:
        return str(a_string)

def coerce_bool(a_string):
    num_bool = bool(coerce(a_string))
    if str(a_string).lower() == "false" or not num_bool:
        return "FALSE"
    elif str(a_string).lower() == "true" or num_bool:
        return "TRUE"


def command(types=(), name=None):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        return typed_func
    return decorate

@command(types=(int, int))
def add(a, b):
    return a + b

@command(types=(int, int))
def sub(a, b):
    return a - b

@command(types=(int, int))
def mult(a, b):
    return a * b

@command(types=(int, int))
def div(a, b):
    return a / b

@command(types=(str,), name="print")  # don't want to overwrite any built-in symbols
def print_cmd(a):
    print(a)

@command(types=())
def eq(a, b):
    return "TRUE" if coerce(a) == coerce(b) else "FALSE"

@command(types=(), name="not")
def not_cmd(x):
    return "TRUE" if coerce(x) == "FALSE" else "FALSE"


def parse(expr):
    curr_expression = []  # the first item will be the name of the operation, any remaining items will be the arguments
    super_expressions = []  # a stack to remember which expression belongs inside which other expression
    curr_token = ""
    touching_parens = False  # Signifying anonymous function calls
    for char in expr:
        # print(f"{curr_expression=}")
        if char == "(":  # got a subexpression
            if touching_parens:
                # Previous complex expression is meant to be applied to the following argument list (which has no operator)
                sub_expression = [curr_expression.pop()]  # Move complex expression to beginning of subexpression
            else:
                sub_expression = [curr_token]  # got operator name
            curr_token = ""
            curr_expression.append(sub_expression)  # sub_expression at current position in current expression's argument list
            super_expressions.append(curr_expression)  # remember where we descended from
            curr_expression = sub_expression  # continue with the subexpression
            touching_parens = False
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)  # got last argument & end of expression
                curr_token = ""
            curr_expression = super_expressions.pop()  # back to last remembered super expression
            touching_parens = True
        elif char in [" ", "\n", "\r"]:
            touching_parens = False
            pass
        elif char == ",":
            if curr_token:
                curr_expression.append(curr_token)  # got argument
                curr_token = ""
            touching_parens = False
        else:
            curr_token += char
            touching_parens = False
    if curr_token:
        curr_expression.append(curr_token)
    return curr_expression

def lookup(name, env):
    value = None
    while env is not None:
        try:
            value = env[0][name]
            return value
        except KeyError:
            env = env[1]
    return name  # If it's not a name, regard expr as a literal (get()'s fallback value)

def bind(args, params, parent_env, update=False):
    sub_env = {}
    for i, param in enumerate(params):
        if param.startswith("*"):  # Support for variable-length arguments
            sub_env[param] = args[i:]
            break
        else:
            sub_env[param] = args[i]
    if update:
        parent_env.update(sub_env)  # Overwrite any names that are present (desired for iteration/tail call optimization)
        return parent_env
    else:
        return (sub_env, parent_env)  # Create a new sub-environment

import time

def evaluate_old(expr, env=None):
    if env is None:
        env = (global_names, None)  # Linked list of tuple(curr env, next env tuple)
    stack = []
    curr_expr = expr
    curr_atoms = []
    curr_op = "expression-list"
    while True:
        time.sleep(0.05)  # Temporary fix for Jupyter's IO-Rate-Exceeded-Error
        if len(stack) > STACK_SIZE:
            raise RuntimeError(f"Exceeded maximum stack size of {STACK_SIZE}.")
        print(f"{curr_expr=}")
        print(f"{curr_atoms=}")
        is_first = not curr_atoms and curr_expr
        all_evaluated = len(curr_atoms) == len(curr_expr)
        if not all_evaluated:
            next_item = curr_expr[len(curr_atoms)]
            print(f"{curr_op=}, {next_item=}, {len(stack)=}")
            if is_first and not isinstance(next_item, list):
                curr_op = next_item
            if curr_op == 'lambda':
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of all args
            elif curr_op == 'set' and len(curr_atoms) == 1:
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of first arg
            elif curr_op == 'if' and len(curr_atoms) >= 2:
                print(f"{curr_op} with cond={bool(curr_atoms[1])} at {len(curr_atoms)}: choosing branch")
                # Rewrite the current expression and replace it with the firing expression
                if coerce_bool(curr_atoms[1]) == "TRUE":
                    new_expr = curr_expr[2]
                else:
                    new_expr = curr_expr[3] if len(curr_expr) >= 4 else "UNKNOWN"
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Throw away stack frame made for if
                curr_expr[len(curr_atoms)] = new_expr  # Make the result directly part of the previous expr level
                print(f"After if: {curr_expr=}, {curr_atoms=}")
            # elif len(curr_atoms) == len(curr_expr)-1   aoseuthoesuteohuo ush !!!!!! and isinstance(curr_atoms[0], list) and curr_atoms[0] and curr_atoms[0][0] == "closure":
            #     print(f"Got closure, evaluating it")
            #     lambda_params = curr_atoms[0][1][0][1:]  # [1:] to split off preceding empty op ''
            #     lambda_body = curr_atoms[0][1][1:]  # Body can contain a series of imperative-style expressions
            #     lambda_captured_env = curr_atoms[0][2]
            #     lambda_args = curr_atoms[1:]
            #     print(f"{lambda_params=}, {lambda_body=}, {lambda_captured_env=}, {lambda_args=}")
            #     # A new stack frame has already been created when entering the closure application and it has saved the previous env.
            #     # So we don't need another stack frame here and can overwrite the env variable:
            #     env = bind(lambda_args, lambda_params, lambda_captured_env)
            #     # Rewrite the current expression to have it applied:
            #     curr_expr = [''] + lambda_body  # Treat the result as a list of expressions, not a single expression/command
            #     curr_atoms = []
            elif isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_op, curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                next_item = lookup(next_item, env)
                # print(f"{next_item=}")
                curr_atoms.append(next_item)
        else:
            # Fully evaluated the expression to apply -- apply it!
            if curr_expr is expr:
                # We're done
                # print(f"{env=}")
                # print("done", len(stack))
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            # We can apply the operator now
            op = curr_atoms[0]
            args = curr_atoms[1:]
            print(f"Applying {op=}, {args=}")
            if isinstance(op, list) and op[0] == "closure":
                print(f"Got closure, evaluating it")
                lambda_params = curr_atoms[0][1][0][1:]  # [1:] to split off preceding empty op ''
                lambda_body = curr_atoms[0][1][1:]  # Body can contain a series of imperative-style expressions
                lambda_captured_env = curr_atoms[0][2]
                lambda_args = curr_atoms[1:]
                print(f"{lambda_params=}, {lambda_body=}, {lambda_captured_env=}, {lambda_args=}")
                # A new stack frame has already been created when entering the closure apply and it has saved the previous env.
                # So we don't need another stack frame here and can overwrite the env variable:
                env = bind(lambda_args, lambda_params, lambda_captured_env)
                # Rewrite the current expression to have it applied:
                curr_expr = [''] + lambda_body  # Treat the result as a list of expressions, not a single expression/command
                curr_atoms = []
                curr_op = "closure"
            else:
                # print(op, args)
                if op == 'lambda':
                    # print(f"Making closure from lambda function")
                    # Create a closure of args (params, *function body), capturing the current env
                    # Note that this is assigned as result and will thus not be evaluated further unless called at a later point.
                    result = ["closure", args, env]
                elif op == 'set':
                    # print(f"Setting {args[0]}={args[-1]}")
                    env[0][args[0]] = args[-1]
                    result = "NOTHING"
                elif op:  # any primitive operator that is not the empty string
                    result = op(args)
                    result = result if result is not None else "NOTHING"
                else:
                    # No operator to apply something to (it's just a list). Leave it as a list (sans empty operator name '')
                    result = curr_atoms[1:]
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Restore previous environment
                # print(f"Returning from {curr_op}")
                # if curr_op == "closure":
                #     curr_op, curr_expr, curr_atoms, env = stack.pop()  # Get rid of the closure layer -- its result is uninterpretable otherwise
                #     # print(f"{curr_atoms=}")
                #     # print(f"Picking last result {result[-1]} from result {result}")
                #     result = result[-1]
                print(f"{curr_op=}, {result=}")
                if curr_op == "closure" and isinstance(result, list):
                    result = result[-1]
                curr_atoms.append(result)

def run(prog):
    ast = parse(prog)
    # print(f"{ast=}")
    return evaluate(ast)


def is_primitive(exp):
    op = exp and not isinstance(exp[0], list) and exp[0]
    return callable(global_names.get(op))


# Try to organize thoughts in a fresh start
def evaluate(expr):
    env = (global_names, None)  # Linked list of tuple(curr env, next env tuple)
    stack = []
    curr_expr = expr
    curr_atoms = []
    curr_task = "eval-expr-list"

    def push():
        print(f"push from {curr_task}")
        stack_frame = (curr_task, curr_expr, curr_atoms, env)
        stack.append(stack_frame)
    
    def pop():
        nonlocal curr_task, curr_expr, curr_atoms, env
        print(f"pop from {curr_task}", end=" ")
        curr_task, curr_expr, curr_atoms, env = stack.pop()
        print(f"to {curr_task}, {curr_expr=}")
    
    # def prepare_primitive_application(curr_item):
    #     nonlocal curr_task, curr_expr, curr_atoms, env
    #     curr_task = "primitive"
    #     curr_atoms = [curr_expr[0]]

    def apply_primitive():
        nonlocal curr_task, curr_expr, curr_atoms, env
        args = curr_atoms[1:]
        op = global_names[curr_expr[0]]
        result = op(args)
        pop()
        curr_atoms.append(result)
        curr_task = "eval-expr"
        
    while True:
        time.sleep(0.05)  # Temporary fix for Jupyter's IO-Rate-Exceeded-Error
        print(f"------\n{curr_task=}")
        print(f"{curr_expr=}")
        print(f"{curr_atoms=}")
        print(f"{len(stack)=}")
        at_expr_start = not curr_atoms
        at_expr_end = len(curr_atoms) == len(curr_expr)
        curr_item = curr_expr[len(curr_atoms)]
        # Wishful thinking -- future me will solve all of these things I'm sure:
        if curr_task == "eval-expr-list":
            print("starting expression")
            push()
            print(f"{curr_expr} = {curr_expr[0]}")
            curr_expr = curr_expr[0]
            curr_atoms = []
            curr_task = "eval-expr"
            continue
        if curr_task == "eval-expr" and is_primitive(curr_item):
            print("starting primitive")
            prepare_primitive_application()
            continue
        if at_expr_end:
            if curr_task == "primitive":
                apply_primitive()
                continue
            elif curr_task == "eval-expr-list":
                result = curr_atoms[-1]  # Ignore all but the last return value in list of expressions
                pop()
                curr_atoms.append(result)
                continue
            elif curr_task == "eval-expr":
                pass
        # On every item
        if not at_expr_end:
            print(f"{curr_item=}")
            curr_atoms.append(lookup(curr_item, env))
        elif not stack and curr_expr is expr:
            return curr_atoms
    
        
        
run("add(1,2) add(3,4)")

------
curr_task='eval-expr-list'
curr_expr=[['add', '1', '2'], ['add', '3', '4']]
curr_atoms=[]
len(stack)=0
starting expression
push from eval-expr-list
[['add', '1', '2'], ['add', '3', '4']] = ['add', '1', '2']
------
curr_task='eval-expr'
curr_expr=['add', '1', '2']
curr_atoms=[]
len(stack)=1
curr_item='add'
------
curr_task='eval-expr'
curr_expr=['add', '1', '2']
curr_atoms=[<function command.<locals>.decorate.<locals>.typed_func at 0x107cef5e0>]
len(stack)=1
curr_item='1'
------
curr_task='eval-expr'
curr_expr=['add', '1', '2']
curr_atoms=[<function command.<locals>.decorate.<locals>.typed_func at 0x107cef5e0>, '1']
len(stack)=1
curr_item='2'
------
curr_task='eval-expr'
curr_expr=['add', '1', '2']
curr_atoms=[<function command.<locals>.decorate.<locals>.typed_func at 0x107cef5e0>, '1', '2']
len(stack)=1


IndexError: list index out of range

[['set', 'l', ['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']]]]
[['l', '3', ['add', '5', '5']]]
[[['l', '3', ['add', '5', '5']], 'a', 'b']]
[['lambda', ['', 'a', 'b'], ['add', 'a', 'b'], ['add', 'b', 'a']], ['', '1', '2']]


In [111]:
l = [1,2,3]
l.pop()
l.append((4,5))
l

[1, 2, (4, 5)]

In [45]:
callable(None)

False

In [114]:


coerce_bool(0)

'FALSE'

In [112]:
NotImplementedError


NotImplementedError

In [103]:
Backup:


STACK_SIZE = 100
global_names = {}

def coerce(a_string):
    try:
        return int(a_string)
    except ValueError:
        pass
    try:
        return float(a_string)
    except ValueError:
        return str(a_string)

def coerce_bool(a_string):
    num_bool = bool(coerce(a_string))
    if str(a_string).lower() == "false" or not num_bool:
        return "FALSE"
    elif str(a_string).lower() == "true" or num_bool:
        return "TRUE"


def command(types=(), name=None):
    def decorate(func):
        nonlocal name
        if name is None:
            name = func.__name__
        def typed_func(args):
            cast_args = [types[i](a) if i < len(types) else a for i, a in enumerate(args)]
            return func(*cast_args)
        global_names[name] = typed_func
        return typed_func
    return decorate

@command(types=(int, int))
def add(a, b):
    return a + b

@command(types=(int, int))
def sub(a, b):
    return a - b

@command(types=(int, int))
def mult(a, b):
    return a * b

@command(types=(str,), name="print")  # don't want to overwrite any built-in symbols
def print_cmd(a):
    print(a)

@command(types=())
def eq(a, b):
    return "TRUE" if coerce(a) == coerce(b) else "FALSE"

def parse(expr):
    curr_expression = []  # the first item will be the name of the operation, any remaining items will be the arguments
    super_expressions = []  # a stack to remember which expression belongs inside which other expression
    curr_token = ""
    touching_parens = False
    for char in expr:
        # print(f"{curr_expression=}")
        if char == "(":  # got a subexpression
            if touching_parens:
                # Previous complex expression is meant to be applied to the following argument list (which has no operator)
                sub_expression = [curr_expression.pop()]  # Move complex expression to beginning of subexpression
            else:
                sub_expression = [curr_token]  # got operator name
            curr_token = ""
            curr_expression.append(sub_expression)  # sub_expression at current position in current expression's argument list
            super_expressions.append(curr_expression)  # remember where we descended from
            curr_expression = sub_expression  # continue with the subexpression
            touching_parens = False
        elif char == ")":
            if curr_token:
                curr_expression.append(curr_token)  # got last argument & end of expression
                curr_token = ""
            curr_expression = super_expressions.pop()  # back to last remembered super expression
            touching_parens = True
        elif char in [" ", "\n", "\r"]:
            touching_parens = False
            pass
        elif char == ",":
            if curr_token:
                curr_expression.append(curr_token)  # got argument
                curr_token = ""
            touching_parens = False
        else:
            curr_token += char
            touching_parens = False
    if curr_token:
        curr_expression.append(curr_token)
    return curr_expression

def lookup(name, env):
    value = None
    while env is not None:
        try:
            value = env[0][name]
            return value
        except KeyError:
            env = env[1]
    return name  # If it's not a name, regard expr as a literal (get()'s fallback value)

def bind(args, params, parent_env):
    sub_env = {}
    for i, param in enumerate(params):
        if param.startswith("*"):  # Support for variable-length arguments
            sub_env[param] = args[i:]
            break
        else:
            sub_env[param] = args[i]
    return (sub_env, parent_env)


def evaluate(expr, env=None):
    if env is None:
        env = (global_names, None)  # Linked list of tuple(curr env, next env tuple)
    stack = []
    curr_expr = expr
    curr_atoms = []
    curr_op = None
    in_closure = None  # TODO: Tail call optimization
    while True:
        if len(stack) > STACK_SIZE:
            raise RuntimeError(f"Max stack size of {STACK_SIZE} exceeded.")
        # print(f"{curr_expr=}")
        # print(f"{curr_atoms=}")
        if len(curr_atoms) == len(curr_expr): # Fully evaluated current expression.
            if curr_expr is expr and curr_op != "if":  # The 'if' part is a stupid hack because we mix expr lists and exprs
                # We're done
                # print(f"{env=}")
                # print("done", len(stack))
                return curr_atoms[-1]  # Last result in imperative-style list of commands counts as return value
            # We can apply the operator now
            op = curr_atoms[0]
            args = curr_atoms[1:]
            if isinstance(op, list) and op[0] == "closure":
                # print(f"Got closure, evaluating it")
                lambda_params = op[1][0][1:]  # split off preceding empty op ''
                lambda_body = op[1][1:]
                lambda_env = op[2]
                # print(f"{lambda_params=}, {lambda_body=}, {lambda_env=}")
                sub_env = bind(args, lambda_params, lambda_env)
                # Save current environment
                stack_frame = ("closure", curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = [''] + lambda_body  # Treat the result as a list, not a command
                curr_atoms = []
                env = sub_env
                # print(stack_frame)
                curr_op = None
            else:
                # print(op, args)
                if op == 'lambda':
                    # print(f"Making closure from lambda function")
                    # Create a closure of args (params, *function body), capturing the current env
                    # Note that this is assigned as result and will thus not be evaluated further unless called at a later point.
                    result = ["closure", args, env]
                elif op == 'set':
                    # print(f"Setting {args[0]}={args[-1]}")
                    env[0][args[0]] = args[-1]
                    result = "NOTHING"
                elif curr_op == 'if':
                    # print(f"if args: {args}")
                    # result = args[1] if coerce_bool(args[0]) == "TRUE" else (args[2] if len(args) > 2 else "NOTHING")
                    # Whatever the last value will be the branch that fired:
                    result = args[1] if len(args) > 0 else "NOTHING"
                elif op:  # any primitive operator that is not the empty string
                    result = op(args)
                    result = result if result is not None else "NOTHING"
                else:
                    # No operator to apply something to (it's just a list). Leave it as a list (sans empty operator name '')
                    result = curr_atoms[1:]
                curr_op, curr_expr, curr_atoms, env = stack.pop()  # Restore previous environment
                # print(f"Returning from {curr_op}")
                if curr_op == "closure":
                    curr_op, curr_expr, curr_atoms, env = stack.pop()  # Get rid of the closure layer -- its result is uninterpretable otherwise
                    # print(f"{curr_atoms=}")
                    # print(f"Picking last result {result[-1]} from result {result}")
                    result = result[-1]
                curr_atoms.append(result)
        else:
            next_item = curr_expr[len(curr_atoms)]
            is_first = not curr_atoms
            if is_first:
                curr_op = next_item
            # print(f"Evaluating {next_item} ({len(curr_atoms)} of {len(curr_expr)-1}) {curr_op=}, {env[0]=}")
            # print(curr_op, len(curr_atoms), curr_atoms)
            if curr_op == 'lambda':
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of all args
            elif curr_op == 'set' and len(curr_atoms) == 1:
                # print(f"{curr_op}, so not evaluating item {len(curr_atoms)}")
                curr_atoms.append(next_item)  # delay evaluation of first arg
            elif curr_op == 'if' and ((len(curr_atoms) == 2 and coerce_bool(curr_atoms[1]) != "TRUE") or
                                      (len(curr_atoms) == 3 and coerce_bool(curr_atoms[1]) == "TRUE")):
                    # print(f"{curr_op} with cond={bool(curr_atoms[1])} at {len(curr_atoms)}, so not evaluating branch")
                    # Completely skip the non-firing branch - we don't need it at all, not even unevaluated
                    # curr_atoms.append(next_item)  # delay evaluation of first arg      
                    curr_expr.pop()  # Re-write the incoming expression
                    pass
            elif isinstance(next_item, list): # Next item is a subexpression.
                # Save current environment
                stack_frame = (curr_op, curr_expr, curr_atoms, env)
                stack.append(stack_frame)
                # Activate a new environment for this subexpression
                curr_expr = next_item
                curr_atoms = []
            else: # Item is already an atom
                next_item = lookup(next_item, env)
                # print(f"{next_item=}")
                curr_atoms.append(next_item)
        print(f"{curr_op=}, {len(stack)=}")

def run(prog):
    ast = parse(prog)
    # print(f"{ast=}")
    return evaluate(ast)


# run("set(l, lambda((a,b) add(a, b) add(b, a)))")
# run("l(3,add(5, 5))")
# run("lambda((a,b) add(a, b) add(b, a))(1,2)")
# run("""
# set(incby, lambda((a), lambda((b) add(b, a))))
# set(plustwo, incby(2))
# plustwo(4)
# """)
# run("""
# set(rec, lambda((x), print(x) rec(add(x, 1))))
# rec(0)
# """)
# run("add(1,3)")
# run("set(a,1), set(b,1) if(eq(a,b), 1, 2)")
run("if(FALSE, 1, 2)")
# run("""
# set(fact, lambda((x) if(eq(x, 1), 1, mult(x, fact(sub(x, 1))))))
# fact(5)
# """)  # 120
# run("""
# set(fact,
#     lambda((x)
#            set(iter, lambda((n, acc)
#                             if(eq(n, 1),
#                                acc,
#                                iter(sub(n, 1), mult(n, acc)))
#                            )
#               ) 
#             iter(x, 1)
#           )
#    )
# fact(6)
# """)  # 120

SyntaxError: invalid syntax (325320499.py, line 1)

Umm, where were we? Got a little sidetracked there.

Ah, yes, I promised some more strange Python stuff.

There are many ways to think about "what should come next" (flow control), and how to bundle chunks of program and make them run at the right time.

Calling functions from other functions is most common -- but it's just one of several ways. It works with a call stack that gets another environment layer (a.k.a. stack frame) pushed on top with each function call (the environment contains the passed arguments and any local variables). That way, we get a fresh, clean environment with only locals in each function, and when we return (and pop the env from the stack), we're right we're we left of in the calling function. But that means that doing real math in a style like inc(inc(inc(...))) would grow to the stack quite significantly and eventually overflow it.

We can be more explicit in what we want to happen next. Continuation-passing style is one of them. There, we give a function that we want to use not only its arguments, but also another function that it should call with the result (we can also pass it continuations for error handling). In the days of pre-Promise (a.k.a. Promise) web dev, something similar was quite common and lovingly called "callback hell" (because for chains of things-to-do-with-results-of-things-to-do, you got {a {lot {of {nested {function {bodies}}}}}}).

Another problem (depending on the language, certainly in Python) is also that when naively calling 1000 consecutive continuations one from another, we'll risk blowing up the stack (even though we don't need it).

There are, however, somewhat more sophisticated ways of handling this type of stuff, even in Python.

In [18]:
# "continuation" test -- unrelated to interpreter, but to weird python

# Helper function do do partial evaluation of functions -- nothing to do with continuations as such
def partial(func, *args, **kwargs):
    def inner(*a, **kwa):
        kwargs.update(kwa)
        return func(*args, *a, **kwargs)
    inner.__name__ = f"partial_{func.__name__}"
    return inner


class Continue(Exception):
    def __init__(self, cont, *args, **kwargs):
        self.cont = cont
        self.args = args
        self.kwargs = kwargs

def add_one(a, cont=None, err=None):
    result = a + 1
    print(f"add_one:   {a=} + 1 = {result} -- {cont}")
    raise Continue(cont, result)  # I don't care whoever called me. Screw that guy and drop the stack: Next stop `cont`!
    return "this never happens :)"

def times_two(b, cont=None, err=None):
    result = b * 2
    print(f"times_two: {b=} * 2 = {result} -- {cont}")
    raise Continue(cont, result)
    return "this never happens, either!"

def divide(x, y, cont=None, err=None):
    try:
        result = x / y
        print(f"divide: {x=} / {y=} = {result}")
    except Exception as e:
        raise Continue(err, f"Error in divide: {e}")
    else:
        raise Continue(cont, result)

# We need a sort of runner to wrap the whole thing. If func is just any normal function, it just runs it and returns its result as usual.
# This is essentially a state machine.
def run(func, *args, **kwargs):
    cont = Continue(func, *args, **kwargs)
    while True:
        try:
            return cont.cont(*cont.args, **cont.kwargs)
        except Continue as c:
            if c.cont:
                cont = c
            else:
                return c.args[0] if c.args else None

def handle_error(e, *args, **kwargs):
    print(f"Umm, gotta go, byeee!\nPS: {e}")

# Now for an (admittedly less-than-beautiful) example
def func():
    func_c = partial(divide, 2, err=handle_error)
    func_b = partial(times_two, cont=func_c)
    func_a = partial(add_one, cont=func_b)
    func_a(1)

run(func)

# In a more standard application of continuation passing, you'd probably catch the signal close to where you called the relevant function.
# It's similar to a callback, but not exactly. The stack is cut off here, while with a callback, the caller's return stack can 
# still be handled in a direct-style fashion.

# Now that you probably wonder about that cool new feature and how it enables clean, beautiful code...

# It doesn't.
# It's GOTO

# Yes, fscking GOTO.
# I'm kidding (a bit), as continuations are actually more than that as they (may) capture state as well (like closures).

add_one:   a=1 + 1 = 2 -- <function partial.<locals>.inner at 0x112a7b790>
times_two: b=2 * 2 = 4 -- <function partial.<locals>.inner at 0x112a7b820>
divide: x=2 / y=4 = 0.5


0.5

I admit that this is not particularly elegant. Here's some syntactic sugar to make it a little more convenient to use.

In [27]:
class Cont():
    def __init__(self, func):
        self.func = func
        self.predecessor = None
        self.successor = None
        self.err_successor = None

    def __rshift__(self, other):  # Chain functions on their success continuation `cont`
        if not isinstance(other, Cont):
            other = Cont(other)
        print(f"{self} >> {other}")
        other.predecessor = self
        self.successor = other
        return other

    def __or__(self, other):  # (success | failure) continuations
        other = other if isinstance(other, Cont) else Cont(other)
        print(f"{self} | {other}")
        if self.predecessor:
            self.predecessor.err_successor = other
        return self

    def run(self, *args, **kwargs):
        first = self
        while first.predecessor is not None:
            first = first.predecessor
        done = False
        cont = Continue(first, *args, **kwargs)
        while not done:
            try:
                cont_obj = cont.cont
                merged_kwargs = {"cont": cont_obj.successor, "err": cont_obj.err_successor}
                merged_kwargs.update(cont.kwargs)
                cont_obj.func(*cont.args, **merged_kwargs)
            except Continue as c:
                if c.cont:
                    cont = c
                else:
                    return c.args[0] if c.args else None
            else:
                done = True

    def __repr__(self):
        return f"Cont({self.func.__name__})"

c = Cont(times_two) >> add_one >> times_two >> partial(divide, 36)

print("-- run first example")
print(c.run(1))

print("------")

c2 = Cont(times_two) >> partial(divide, 3) >> add_one | handle_error

print("-- run second example")
print(c2.run(0))

Cont(times_two) >> Cont(add_one)
Cont(add_one) >> Cont(times_two)
Cont(times_two) >> Cont(partial_divide)
-- run first example
times_two: b=1 * 2 = 2 -- Cont(add_one)
add_one:   a=2 + 1 = 3 -- Cont(times_two)
times_two: b=3 * 2 = 6 -- Cont(partial_divide)
divide: x=36 / y=6 = 6.0
6.0
------
Cont(times_two) >> Cont(partial_divide)
Cont(partial_divide) >> Cont(add_one)
Cont(add_one) | Cont(handle_error)
-- run second example
times_two: b=0 * 2 = 0 -- Cont(partial_divide)
Umm, gotta go, byeee!
PS: Error in divide: division by zero
None


## Generators and Pipelines

In [365]:
# So I want to make my own counter

def count():
    i = 0
    return i + 1

count()
count()
count()

1

In [387]:
# Hmm, we need some kind of state, but don't want the inc(inc(...)) weirdness from earlier.
# We might be tempted to write a class, but it's not necessary:

def make_counter(i=0):
    def nxt():
        nonlocal i
        i += 1
        return i
    return nxt

ctr = make_counter()
ctr()  # Calling it gets the next value
ctr()
ctr()

3

In [413]:
# With this bare-bones technique, we can do quite weird things, all without using any special Python stuff like Iterators or such.
# Here's a number-processing ... thing ... that alternatingly multiplies and adds numbers to a start value,,
# for no particular reason whatsoever:

def make_whatever(inputs, accumul=1):
    def mult():
        nonlocal accumul
        accumul *= inputs()
        return accumul
    def add():
        nonlocal accumul
        accumul += inputs()
        return accumul
    next_op = add
    def nxt():
        nonlocal next_op
        next_op = mult if next_op is add else add
        return next_op()
    return nxt

wut = make_whatever(make_counter())  # The nesting is also a mini pipeline at the same time
[(i, str(wut())) for i in range(1,10)]

[(1, '1'),
 (2, '3'),
 (3, '9'),
 (4, '13'),
 (5, '65'),
 (6, '71'),
 (7, '497'),
 (8, '505'),
 (9, '4545')]

In [379]:
# Of course, Python has Iterators (things that iterate over data by understanding the next() message), specifically range(), which is
# actually a Generator. A Generator is a type of iterator that does not need a full data structure, but executes code
# on each iteration. This is a Generator:

def make_counter(i=0):
    while True:
        i += 1
        yield i

count = make_counter()
next(count)
next(count)
next(count)

3

In [380]:
# Yield is used instead of return here. Yield keeps our generator's stack intact when "returning" to the caller.
# So the next time we call next(count) it will continue where we left off:

next(count)
# 4

# I won't bore you with terminology about Coroutines and Continuations, even though they may be relevant here.
# Just as our DIY counter above, generators keep the memory usage constant, even with (theoretically) infinite data.

# Generators can of course use other generators as their input. They be written as comprehensions:
squ = (a ** 2 for a in make_counter())

next(squ)
next(squ)
next(squ)
next(squ)

16

In [374]:
# While Generators are often used for iterating, the internal algorithm does not need to be iterative at all:
def bla():
    yield 1
    yield 2
    return 3

for b in bla():
    print(b)

1
2


In [434]:
# -- Every yield emits a value. Note that the return value is ignored. Return is just another way to stop the generation.

# Generators are essentially streams.
# So they can be used for signal processing (pipeline-like data stream manipulation with lazy evaluation (no data "slurping"))

def make_story():
    p = "Once upon a time there was a story that began:".split()
    while True:
        yield p[0]
        p = p[1:] + p[:1]

def remove_vowels(source):
    for s in source:
        yield "".join(c for c in str(s) if c.lower() not in "aeiou")

def up_low(source):
    up = True
    for s in source:
        yield s.upper() if up else s.lower()
        up = not up

def ignore_empty(source):
    for s in source:
        if s:
            yield s

s = make_story()
rv = remove_vowels(s)
ul = up_low(rv)
ie = ignore_empty(ul)
print(" ".join(s for _, s in zip(range(30), ie)))

# Which is equivalent to the rather unreadable up_low(remove_vowels(make_story())):
print(" ".join(
    s for _, s in zip(range(30), 
                      ignore_empty(up_low(remove_vowels(make_story())))
                     )
))

# (Exercise: write a simple low-pass filter for a stream of numbers)
# More Info: https://scipy-lectures.org/advanced/advanced_python/

NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry
NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry THT bgn: NC pn tm THR ws stry


In [433]:
# Note that in the generator example above, every processing step refers to its source.
# This is a "pull" pipeline: Trying to get a value out of the last element in the chain makes it get one from
# the element before, and so on.
# This stream processing is called lazy.

# Generators also support "push" semantics. In that, the source decides when to push something into the beginning
# of the pipeline, which makes all the following elements take a processing step.
# This processing is based on an event stream.

def make_input(target):
    print("Enter some words, press enter after each word, and just enter to stop")
    next(target)  # initialize target
    while True:
        s = input()
        if not s:
            break
        target.send(s)
    print("Thank you for using make_input!")

def remove_vowels2(target):
    next(target)
    while True:
        s = yield
        target.send("".join(c for c in s if c.lower() not in "aeiou"))

def up_low2(target):
    next(target)
    up = True
    while True:
        s = yield
        target.send(s.upper() if up else s.lower())
        up = not up

def ignore_empty2(target):
    next(target)
    while True:
        s = yield
        if s:
            target.send(s)

def emitter():
    while True:
        print(f" --> {(yield)}")


make_input(
    remove_vowels2(
        up_low2(
            ignore_empty2(
                emitter()
))))
# Note that, somewhat surprisingly, the nested function calls are in a left-to-right-readable order.

# The previous variant of writing it is now actually less readable:
# ul2 = up_low2(emitter())
# rv2 = remove_vowels2(ul2)
# make_input(rv2)


# For clarity, I'm omitting some stuff that's not really needed here but might be useful with other resources,
# like closing the targets and catching GeneratorExit to clean up/close the following chain.


Enter some words, press enter after each word, and just enter to stop


 hello


 --> HLL


Thank you for using make_input!


In [438]:
# Come to think of it, we might be able to have our cake and eat it, too!
# That is, use our previously defined pull-type generators with a push-type source:

def push_to_pull_adapter():
    pushed_val = None
    while True:
        pushed_val = yield pushed_val

def consumer(source):
    for s in source:
        print(s)

pull_source = push_to_pull_adapter()
# chain = consumer(ignore_empty(up_low(remove_vowels(pull_source))))
# pushing_source = make_input(pull_source)

# Darn. I commented things out because I didn't want to crash your Kernel.
# This creates an infinite loop. The reason is that my_iter.send(None) and next(my_iter) are the same thing
# and any of those two makes the generator continue at the yield statement.
# So anything pulling at it is calling next() and so de facto calling .send(None) all the time, making
# it go on and on with None values.

# There's a thinking error in this, too: Which side is the user code on? If it's the pushing code, then
# we cannot "start" the pulling code, and the other way round. At least not without threads (green or other).

# There's a pattern called Transducers, which solves this problem.

## Puzzling Generators!

In [445]:
# We already know that you can build an interpreter using simple first-class functions that just do stuff and return it.

puzzle = """
E -> G
B -> X
S -> A
S -> D
A -> B
A -> C
X -> Y
C -> G
B -> A
B -> E
B -> C
D -> G
"""

transitions = [(a, b) for a, _, b in [line.split() for line in puzzle.split("\n") if line]]

def find_path(curr, goal, transitions, path=None):
    if path == None:
        path = [curr]
    if curr == goal:
        return path
    neighbors = [b for a, b in transitions if a == curr]
    for new_neighbor in [n for n in neighbors if n not in path]:
        print(f"Trying {curr} -> {new_neighbor}, {path=}")
        result = find_path(new_neighbor, goal, transitions, path + [new_neighbor])
        if result is not None:
            return result
    return None  # being explicit

find_path("S", "G", transitions)

Trying S -> A, path=['S']
Trying A -> B, path=['S', 'A']
Trying B -> X, path=['S', 'A', 'B']
Trying X -> Y, path=['S', 'A', 'B', 'X']
Trying B -> E, path=['S', 'A', 'B']
Trying E -> G, path=['S', 'A', 'B', 'E']


['S', 'A', 'B', 'E', 'G']

In [448]:
# So we made a simple problem solver with backtracking (depth-first tree search)

# What if we want to explore more than one solution? Do we have to store them in a list structure? Pshaw!
# We don't need no stinking temporary data structures. We've got Generators.

def solutions(curr, goal, transitions, path=None):
    if path == None:
        path = [curr]
    if curr == goal:
        yield path
    neighbors = [b for a, b in transitions if a == curr]
    for new_neighbor in [n for n in neighbors if n not in path]:
        print(f"Trying {curr} -> {new_neighbor}, {path=}")
        yield from solutions(new_neighbor, goal, transitions, path + [new_neighbor])

for i, p in enumerate(solutions("S", "G", transitions)):
    print(f"Solution {i+1}: {p}")

Trying S -> A, path=['S']
Trying A -> B, path=['S', 'A']
Trying B -> X, path=['S', 'A', 'B']
Trying X -> Y, path=['S', 'A', 'B', 'X']
Trying B -> E, path=['S', 'A', 'B']
Trying E -> G, path=['S', 'A', 'B', 'E']
Solution 1: ['S', 'A', 'B', 'E', 'G']
Trying B -> C, path=['S', 'A', 'B']
Trying C -> G, path=['S', 'A', 'B', 'C']
Solution 2: ['S', 'A', 'B', 'C', 'G']
Trying A -> C, path=['S', 'A']
Trying C -> G, path=['S', 'A', 'C']
Solution 3: ['S', 'A', 'C', 'G']
Trying S -> D, path=['S']
Trying D -> G, path=['S', 'D']
Solution 4: ['S', 'D', 'G']


In [449]:
# So, yeah, those transitions from one state to another are nice, but isn't the world more complex, I hear you ask.
# Well, yes and no. The truth is that pretty much any constraint problem can be reduced to a problem of transitions.

maze = """
########### #
# #   #     #
# # #   ### #
#    ###    #
## #     ####
## ##########
"""

def get_transitions(maze):
    transitions = []
    rows = [r.strip() for r in maze.split("\n") if r.strip()]
    nrows = len(rows)
    ncols = len(rows[0])
    for y, row in enumerate(rows):
        for x, char in enumerate(row):
            if char == "#":
                continue
            here = (x, y)
            if x < ncols-1 and row[x+1] != "#":
                transitions.append((here, (x+1, y)))
            if x > 1 and row[x-1] != "#":
                transitions.append((here, (x-1, y)))
            if y < nrows-1 and rows[y+1][x] != "#":
                transitions.append((here, (x, y+1)))
            if y > 1 and rows[y-1][x] != "#":
                transitions.append((here, (x, y-1)))
    print(f"Found {len(transitions)} transitions in {ncols} by {nrows} maze: {transitions}")
    return transitions

def vis_path(path):
    for y, row in enumerate([r.strip() for r in maze.split("\n") if r.strip()]):
        row = "".join(["o" if (x, y) in path else c for x, c in enumerate(row)])
        print(row)

transitions = get_transitions(maze)
path = None
for i, p in enumerate(solutions((11,0), (2,5), transitions)):
    print(f"Solution {i+1}: {p}")
    vis_path(p)
    print("----------------")


Found 61 transitions in 13 by 6 maze: [((11, 0), (11, 1)), ((1, 1), (1, 2)), ((3, 1), (4, 1)), ((3, 1), (3, 2)), ((4, 1), (5, 1)), ((4, 1), (3, 1)), ((5, 1), (4, 1)), ((5, 1), (5, 2)), ((7, 1), (8, 1)), ((7, 1), (7, 2)), ((8, 1), (9, 1)), ((8, 1), (7, 1)), ((9, 1), (10, 1)), ((9, 1), (8, 1)), ((10, 1), (11, 1)), ((10, 1), (9, 1)), ((11, 1), (10, 1)), ((11, 1), (11, 2)), ((1, 2), (1, 3)), ((1, 2), (1, 1)), ((3, 2), (3, 3)), ((3, 2), (3, 1)), ((5, 2), (6, 2)), ((5, 2), (5, 1)), ((6, 2), (7, 2)), ((6, 2), (5, 2)), ((7, 2), (6, 2)), ((7, 2), (7, 1)), ((11, 2), (11, 3)), ((11, 2), (11, 1)), ((1, 3), (2, 3)), ((1, 3), (1, 2)), ((2, 3), (3, 3)), ((2, 3), (1, 3)), ((2, 3), (2, 4)), ((3, 3), (4, 3)), ((3, 3), (2, 3)), ((3, 3), (3, 2)), ((4, 3), (3, 3)), ((4, 3), (4, 4)), ((8, 3), (9, 3)), ((8, 3), (8, 4)), ((9, 3), (10, 3)), ((9, 3), (8, 3)), ((10, 3), (11, 3)), ((10, 3), (9, 3)), ((11, 3), (10, 3)), ((11, 3), (11, 2)), ((2, 4), (2, 5)), ((2, 4), (2, 3)), ((4, 4), (5, 4)), ((4, 4), (4, 3)), ((5

To guarantee to get the shortest solution first, we would have to change `solutions` to use breadth-first search
instead of depth-first search. The downside is that breadth-first search potentially uses much more memory.

This is left as an exercise to the reader. :)

## Function Overloading

Other programming languages support a type of ad-hoc polymorphism called function overloading.

This makes it possible to use the same function to perform an operation on different numbers of arguments and different data types.

For example, a function "add(a, b)" could be implemented so it supports the addition of numbers, joining strings, appending lists, etc.
This is particularly useful if the operation has semantic similarities in those data types (that differ only in the concrete implementation), e.g. there is an empty element, an identity operation, and the transformation of one argument of the type to a result of the same type, but modified by another argument.

While Python does not support function overloading, it does support polymorphism through classes and objects: You can define different classes (or subclasses) like "RationalNumber" and "ComplexNumber" that have methods of the same name, so users can just use them without having to know which object they are dealing with. To make this simpler, many Python operators like `=`, `+` etc. have special names that you can use in your classes: `__eq__`, `__add__`.

If you don't need (or want) to write classes, Python does not have your back this time.

But we can simulate function overloading with our own function like this:

In [349]:
import inspect
from typing import Callable

def divfloat(a: float, b: float):
    print("Called float variant")
    return a / b

def divint(a: int, b: int):
    print("Called int variant")
    return int(a / b)  # VERY strict int variant :)

def divtwo(a):
    print("Called one-param variant")
    return a / 2

def overload(*funcs):
    dispatch = {}
    for func in funcs:
        params = list(inspect.signature(func).parameters.keys())
        types = [func.__annotations__.get(p) for p in params]
        key = ";".join((str(t) for t in types))
        dispatch[key] = func  # All-in-one lookup for both arity and types (could be better...)

    if len(funcs) != len(dispatch):
        raise TypeError(f"Overloaded functions should have different signatures: {funcs}")
    
    def inner(*args):
        type_strs, arg_types = zip(*[(str(type(arg)), type(arg)) for arg in args])
        for try_num in range(len(args)+1):
            key = ";".join(type_strs[:(len(args)-try_num)] + ("None",)*try_num)
            func = dispatch.get(key)
            if func is not None:
                return func(*args)
        raise TypeError(f"No matching function with signature {key}")

    return inner

div = overload(divint, divfloat, divtwo)
print(f"{div(8, 2)=}")
print(f"{div(3, 2)=}")
print(f"{div(3.0, 2.0)=}")
print(f"{div(3)=}")
print(f"{div(3.0)=}")

Called int variant
div(8, 2)=4
Called int variant
div(3, 2)=1
Called float variant
div(3.0, 2.0)=1.5
Called one-param variant
div(3)=1.5
Called one-param variant
div(3.0)=1.5


In [310]:
params = list(inspect.signature(divint).parameters.keys())
types = [divint.__annotations__.get(p) for p in params]
print(";".join([str(t) for t in types]))
strs, types = zip(*((str(type(arg)), type(arg)) for arg in [1,"a"]))
print(";".join(strs))
# (None,)*3
[1,2,3][:2]

<class 'int'>;None
<class 'int'>;<class 'str'>


[1, 2]

## Transducers
You probably already know higher-function orders like map, filter and reduce.
In case you don't, let's make them (they are easy).

BTW: Python functools offers a ton of related functionality.

In [23]:
some_stream = [1,2,3,4,5,6]  # could be any iterable, generator, etc.

def my_map(func, data):
    for d in data:
        yield func(d)

print(f"{list(my_map(lambda x: x * 2, some_stream))=}")

def my_filter(predicate, data):
    for d in data:
        if predicate(d):
            yield d

print(f"{list(my_filter(lambda x: x % 2 == 0, some_stream))=}")

def my_reduce(func, aggregator, data):
    for d in data:
        aggregator = func(aggregator, d)
    return aggregator

print(f"{my_reduce(lambda a, x: a + x, 0, some_stream)=}")

list(my_map(lambda x: x * 2, some_stream))=[2, 4, 6, 8, 10, 12]
list(my_filter(lambda x: x % 2 == 0, some_stream))=[2, 4, 6]
my_reduce(lambda a, x: a + x, 0, some_stream)=21


Some higher-order functions like these have interesting guarantees. For example, map and filter can be carried out in parallel. If they implement this, it can basically be offered "for free" to the user.

It's important to note that these functions are not specific to arithmetics or even single values. They can also process other data.

You can also compose (combine) them. You can however only use them on iterables, as shown below.

In [69]:
# Combining higher-order functions by nesting them:

print(f"{list(my_map(lambda x: x * 2, my_filter(lambda x: x % 2 == 0, some_stream)))=}")

# Helper function to create a nested composed function from a list of functions (does not execute them):

def comp(*functions):
    def inner(*args, **kwargs):
        result_so_far = identity(*args, **kwargs)
        for func in reversed(functions):  # The reversing makes sense later -- for now it mimics the nesting order
            result_so_far = func(*args, **kwargs)
            args = (result_so_far, )
            kwargs.clear()
        return result_so_far
    return inner

def identity(*args, **kwargs):
    """Identity transformation: output equals input (first argument). If no input is given, returns None."""
    return None if not args else args[0]

# Also a little partial function application helper (exists in functools, too, of course)
def partial(func, *args, **kwargs):
    def inner(*a, **kwa):
        return func(*args, *a, **kwargs, **kwa)
    return inner

square_and_inc = comp(lambda x: x + 1, lambda x: x * x)
print(f"{square_and_inc(3)=}")

# And now, as promised, for higher-order functions:
even_squared = comp(partial(my_map, lambda x: x * 2),
                    partial(my_filter, lambda x: x % 2 == 0))

print(f"`even_squared` is a function: {even_squared}")
print(f"{list(even_squared(some_stream))=}")

# Having to use `partial` in order to get pre-configured step functions for mapping and filtering is somewhat clumsy.
# We'll find a common function signature to avoid this later. It will be generic enough that we also won't need generators then.

list(my_map(lambda x: x * 2, my_filter(lambda x: x % 2 == 0, some_stream)))=[4, 8, 12]
square_and_inc(3)=10
`even_squared` is a function: <function comp.<locals>.inner at 0x1114d4e50>
list(even_squared(some_stream))=[4, 8, 12]


Transducers can be seen as a generalization of these higher-order functions.

Except they're not (they are more). ;) As we see further down, we can use transducers for non-iterables as well.

When we ran map, filter and reduce with our lambda functions as parameter, we combined the functionality of the higher-order function with the lambda.

Transducers make this composition of functionalities explicit.

Unfortunately, it is not a true generalization concerning state and therefore concurrency. Because the reduce case cannot be parallelized, there is no guarantee that every transducer can be parallelized. Reduce also changes the shape of the output. That's why it's usually handled as a special case (optionally) at the end of the transducer chain.

In [353]:
# Transducers, as they have been popularized by Clojure, do not need something like generators at all:

def filtering(pred):  # Creates a filtering transducer with a particular predicate
    def transducer(next_reducer):  # Interface to chain the current transducer/reducer with the next one
        def reducer(*args):  # A reducer may (does not have to) aggregate values. It can also just transform values.
            """args := [result_so_far, [input]]"""
            # Python would benefit from overloading. Unfortunately we have to simulate it here.
            if len(args) == 0:  # Init case. No arguments: Produce the identity
                return None  # or identity function?
            elif len(args) == 1:  # Completion case. Got signalled to end: return result_so_far (reduction?)
                return args[0]
            elif len(args) == 2:  # Step case. Doing our thing. Here: filtering.
                if pred(args[1]):
                    return next_reducer(*args)
                else:
                    return args[0]
            else:
                raise ValueError(f"Reducer expects 0, 1 or 2 arguments. Got {len(args)}: {args}")
        return reducer
    return transducer

is_odd = lambda x: x % 2 == 1

# One-time use example: create a transducer filtering odd numbers, that as a next step sums up the results, and let it loose on 5:
filtering(is_odd)(lambda r, x: r + x)(0, 5)

# We see that every part is configurable: We can separate the filter predicate from the reducing function and from the application,
# thereby being flexible in the data type of the elements, as well as the kind of data structure of the result (reduction).

# Command to apply it to data from an iterator (which, remember, can also be a generator)
def transduce(transducer, reducer, coll):
    ...

And here is a nice video from a guy with a strangely familiar accent explaining how transducers work in a language called Clojure:
https://www.youtube.com/watch?v=TaazvSJvBaw

## Propagators

What they are:
https://www.youtube.com/watch?v=nY1BCv3xn24

In [29]:
NOTHING = "*NOTHING*"  # Generic for all cell classes

class CellError(Exception):
    pass

class Cell:  # Simple non-contradictory cell. There can be other types of cells (-> join-semilattice)
    num = 0
    CONTRADICTION = "*CONTRADICTION*"  # Specific per cell class
    
    def __init__(self, debug_name=""):
        self._value = NOTHING
        self._out_propagators = []
        self._debug_name = debug_name

    def _merge(self, new_information):
        old_information = self._value
        if new_information is NOTHING or old_information == new_information:
            return old_information
        elif old_information is NOTHING:
            return new_information
        else:
            return Cell.CONTRADICTION

    def write(self, value):
        merged_value = self._merge(value)
        if merged_value != self._value:
            self._value = merged_value
            for p in self._out_propagators:
                p.notify()

    def value(self):
        if self._value is Cell.CONTRADICTION:
            return CellError(Cell.CONTRADICTION)
        return self._value

    def register(self, propagator):
        self._out_propagators.append(propagator)
        # propagator.notify()  # Don't notify here to make sure everything is connected first (may change when scheduler is implemented)

    def __repr__(self):
        return f"Cell({self._value!r}){f'<{self._debug_name}>' if self._debug_name else ''}"


class Propagator:
    def __init__(self, func, *cells, optional=None):
        """Create a propagator with 0..N input cells and 1 output cell.
        `optional`: iterable with input cells not needed for firing.
        """
        self._func = func
        self._inputs = cells[:-1]
        self._optional_inputs = [] if optional is None else optional
        if any([i not in self._inputs for i in self._optional_inputs]):
            raise ValueError("Optional input cells must also be specified as cell parameters.")
        self._output = cells[-1]
        for cell in self._inputs:
            cell.register(self)
        self.notify()  # Do notification here to make sure everything is connected first (may change when scheduler is implemented)

    def notify(self):
        required = [c for c in self._inputs if c not in self._optional_inputs]
        for c in required:
            val = c.value()
            if val is NOTHING:
                return  # Only fire if all required are available
            if isinstance(val, CellError):
                self._output.write(val.args[0])  # Propagate error directly
                return
        result = self._func(*[c.value() for c in self._inputs])
        # print(f"{self._func}({self._inputs}) --> {result} (to {self._output})")
        self._output.write(result)

    def __repr__(self):
        return f"Propagator({self._func.__name__}, {', '.join([str(i) for i in self._inputs])}, {self._output})"


In [30]:
cA = Cell()
cB = Cell()
cC = Cell()

# cA + cB = cC

p = Propagator(lambda a,b: a+b, cA, cB, cC)

print(cC)
cA.write(3)
print(cC)
cB.write(2)
print(cC)
cB.write(1)
print(cC)

Cell('*NOTHING*')
Cell('*NOTHING*')
Cell(5)
Cell('*CONTRADICTION*')


In [31]:
cA = Cell()
cB = Cell()
cC = Cell()

# We can add connections in any direction, here to reverse the addition:
# cA + cB = cC =>
#   cC - cA = cB,
#   cC - cA = cB

Propagator(lambda a,b: a+b, cA, cB, cC)
Propagator(lambda c,a: c-a, cC, cA, cB)
Propagator(lambda c,b: c-b, cC, cB, cA)

print(cB)
cA.write(3)
print(cB)
cC.write(2)
print(cB)

Cell('*NOTHING*')
Cell('*NOTHING*')
Cell(-1)


In [32]:
# If we want, we can separate the creation of the propagator proper from attaching it to cells

def PropagatorTemplate(func):
    def make_attached_propagator(*cells):
        return Propagator(func, *cells)
    return make_attached_propagator

In [33]:
cA = Cell()
cB = Cell()
cC = Cell()

attach_adder = PropagatorTemplate(lambda a,b: a+b)
attach_subtractor = PropagatorTemplate(lambda a,b: a-b)

attach_adder(cA, cB, cC)
attach_subtractor(cC, cA, cB)
attach_subtractor(cC, cB, cA)

print(cA)
cB.write(3)
print(cA)
cC.write(2)
print(cA)

Cell('*NOTHING*')
Cell('*NOTHING*')
Cell(-1)


We can further package multi-propagator concepts like "addition" to make them more reusable:

In [34]:
# They don't need to work forwards and backwards, but we'll do make them to show off a little.
# (Radul/Sussman call bijective propagator networks "constraints". So these are constraints)
def addition(a, b, c):
    Propagator(lambda a,b: a+b, a, b, c)
    Propagator(lambda c,a: c-a, c, a, b)
    Propagator(lambda c,b: c-b, c, b, a)

def subtraction(a, b, c):
    addition(c, b, a)

def multiplication(a, b, c):
    Propagator(lambda a,b: a*b, a, b, c)
    Propagator(lambda c,a: c/a, c, a, b)
    Propagator(lambda c,b: c/b, c, b, a)

def division(a, b, c):
    multiplication(c, b, a)

u = Cell()
v = Cell()
w = Cell()
x = Cell()
y = Cell()
z = Cell()
c = Cell()

# (u - v) * (x + y) = c
#    w        z     = c
subtraction(u, v, w)
addition(x, y, z)
multiplication(w, z, c)

# Smoke test:
# u.write(3)
# v.write(1)
# print(w)
# x.write(5)
# y.write(2)
# print(z)
# print(c)
# ---
# Cell(2)
# Cell(7)
# Cell(14)

# Simple reversal test:
# c.write(14)
# w.write(2)
# print(z)
# ---
# Cell(7.0)

# Now for the furious finale: solving the equation (sort of)
c.write(14)
v.write(1)
z.write(7)
print(u, w, x)
# Cell(3.0) Cell(2.0) Cell(None) -- We already know u from v and w (which we know from z and c), but don't know x yet
y.write(2)
print(u, w, x)
# Cell(3.0) Cell(2.0) Cell(5) -- Now we know everything

Cell(3.0) Cell(2.0) Cell('*NOTHING*')
Cell(3.0) Cell(2.0) Cell(5)


Now for some less mathematical propagators (cf. Radul/Sussman: The Art of the Propagator)

In [35]:
def conditional(predicate, if_true, if_false, output):
    # Propagator(lambda p, t, f: print(f"if {p, t, f} --> {t if p else f}") or t if p else f, 
    Propagator(lambda p, t, f: t if p else f, 
               predicate, if_true, if_false, output,
               optional=[if_true, if_false])  # Only predicate is strictly needed
    # We can also derive (partial) information from the inverse (usable as constraints)
    # a -> b  =>  -b -> -a
    Propagator(lambda o, t: False if t is not NOTHING and o != t else NOTHING, output, if_true, predicate, optional=[if_true])
    Propagator(lambda o, f: True if f is not NOTHING and o != f else NOTHING, output, if_false, predicate, optional=[if_false])
    # Propagator(lambda o, t: print(f"inv_if1 {o, t} --> {False if t is not NOTHING and o != t else NOTHING}") or False if t is not NOTHING and o != t else NOTHING, output, if_true, predicate, optional=[if_true])
    # Propagator(lambda o, f: print(f"inv_if2 {o, f} --> {True if f is not NOTHING and o != f else NOTHING}") or True if f is not NOTHING and o != f else NOTHING, output, if_false, predicate, optional=[if_false])

def switch(predicate, if_true, output):
    conditional(predicate, if_true, Cell(), output)

def constant(value, output):
    Propagator(lambda: value, output)

def equality(a, b, output):
    # Propagator(lambda a, b: print(f"{a}=={b} --> {a==b}") or a == b, a, b, output)
    # Propagator(lambda o, a: print(f"eq_inv1 {o,a} --> {a if o else NOTHING}") or a if o else NOTHING, output, a, b)
    # Propagator(lambda o, b: print(f"eq_inv2 {o,b} --> {b if o else NOTHING}") or b if o else NOTHING, output, b, a)
    Propagator(lambda a, b: a == b, a, b, output)
    Propagator(lambda o, a: a if o else NOTHING, output, a, b)
    Propagator(lambda o, b: b if o else NOTHING, output, b, a)

def negation(a, output):
    Propagator(lambda a: not a, a, output)
    Propagator(lambda o: not o, output, a)

a = Cell()
b = Cell()
a_is_even = Cell()
result = Cell()

constant(77, b)
print(f"{a=}, {b=}, {a_is_even=}, {result=}")

Propagator(lambda a: a % 2 == 0, a, a_is_even)

conditional(a_is_even, a, b, result)

a.write(8)
print(f"{a=}, {b=}, {a_is_even=}, {result=}")

b.write(99)  # Create a contradiction
# Note that because if_true/if_false are optional,
# nothing blows up even if b (=if_false) is contradictory
print(f"{a=}, {b=}, {a_is_even=}, {result=}")


a=Cell('*NOTHING*'), b=Cell(77), a_is_even=Cell('*NOTHING*'), result=Cell('*NOTHING*')
a=Cell(8), b=Cell(77), a_is_even=Cell(True), result=Cell(8)
a=Cell(8), b=Cell('*CONTRADICTION*'), a_is_even=Cell(True), result=Cell(8)


In [36]:
# Trying out inverse conditionals
raining = Cell()
street_wet = Cell()
const_true = Cell()

constant(True, const_true)
switch(raining, const_true, street_wet)

print(raining, const_true, street_wet)
# TODO: if I set if_true/if_false to optional, this blows up, and if I don't cutting off branches above does not work
street_wet.write(False)
print(raining, const_true, street_wet)  # If the consequent is false, we know that the antecedent must be false

print("----")

bla = Cell()
are_equal = Cell()
print(street_wet, bla, are_equal) 
equality(raining, bla, are_equal)
print(street_wet, bla, are_equal) 
are_equal.write(True)
print(street_wet, bla, are_equal)  # We can deduct bla == False from the knowledge that equality holds

Cell('*NOTHING*') Cell(True) Cell('*NOTHING*')
Cell(False) Cell(True) Cell(False)
----
Cell(False) Cell('*NOTHING*') Cell('*NOTHING*')
Cell(False) Cell('*NOTHING*') Cell('*NOTHING*')
Cell(False) Cell(False) Cell(True)


Let's try our hand at recursion

In [37]:
num_steps = Cell()
last_val = Cell()
curr_val = Cell()

Propagator(lambda last, curr: last + curr, last_val, curr_val, last_val)

last_val.write(0)
curr_val.write(1)
print(last_val)

Cell('*CONTRADICTION*')


Ah, it would have been too easy -- of course we cannot overwrite cells, just "add information" to them. With the current type of cells, we can do it only once (others are possible). So, for now, we have to somehow work around it:
- Looks like we have to create new cells for every recursion/iteration step
- We also have to count down the number of steps still needed
- Only if it's zero do we write into a result cell (exactly once)

In [38]:
def fib_step(n, last, curr, new_curr, new_n, result):
    # zero = Cell()  # We'll use the walrus operator to create a variable at the same time as passing its value
    constant(0, zero:=Cell())
    equality(n, zero, is_done:=Cell())
    switch(is_done, curr, result)
    # Note that, although this looks "else-ish", *all* of this code will be executed at all times.
    # We control the information flow, though, to make propagators fire selectively.
    negation(is_done, not_done:=Cell())
    switch(not_done, last, last_if_not_done:=Cell())
    switch(not_done, curr, curr_if_not_done:=Cell())
    addition(last_if_not_done, curr_if_not_done, new_curr)
    constant(1, one:=Cell())
    switch(not_done, one, one_if_not_done:=Cell())
    subtraction(n, one_if_not_done, new_n)


fib_step(num_steps:=Cell(), val1:=Cell(), val2:=Cell(), new_val:=Cell(), new_n:=Cell(), result:=Cell())

print(num_steps, val1, val2, new_val, new_n, result)
val1.write(0)
val2.write(1)
num_steps.write(2)
print(num_steps, val1, val2, new_val, new_n, result)
fib_step(num_steps:=new_n, val1:=val2, val2:=new_val, new_val:=Cell(), new_n:=Cell(), result:=Cell())
print(num_steps, val1, val2, new_val, new_n, result)
fib_step(num_steps:=new_n, val1:=val2, val2:=new_val, new_val:=Cell(), new_n:=Cell(), result:=Cell())
print(num_steps, val1, val2, new_val, new_n, result)
fib_step(num_steps:=new_n, val1:=val2, val2:=new_val, new_val:=Cell(), new_n:=Cell(), result:=Cell())
print(num_steps, val1, val2, new_val, new_n, result)
fib_step(num_steps:=new_n, val1:=val2, val2:=new_val, new_val:=Cell(), new_n:=Cell(), result:=Cell())
print(num_steps, val1, val2, new_val, new_n, result)


Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*')
Cell(2) Cell(0) Cell(1) Cell(1) Cell(1) Cell('*NOTHING*')
Cell(1) Cell(1) Cell(1) Cell(2) Cell(0) Cell('*NOTHING*')
Cell(0) Cell(1) Cell(2) Cell('*NOTHING*') Cell('*NOTHING*') Cell(2)
Cell('*NOTHING*') Cell(2) Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*')
Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*')


In [39]:
def compound_propagator(prop_building_func, *input_cells):
    """Creates a propagator that executes prop_building_func only once, when any input becomes non-nothing"""
    constructed = False
    def check_and_build(*cells):
        nonlocal constructed
        if not constructed and any(c is not NOTHING for c in cells[:-1]):
            prop_building_func()
            constructed = True
    # Make sure the compound propagator is triggered for available inputs:
    Propagator(check_and_build, *input_cells, Cell())

def fib_iter(n, last, curr, new_curr, new_n, result):
    def prop_building_func():
        fib_step(n, last, curr, new_curr, new_n, result)
        fib_iter(new_n, curr, new_curr, Cell(), Cell(), result)
    compound_propagator(prop_building_func, last, curr)


fib_iter(num_steps:=Cell(), val1:=Cell("val1"), val2:=Cell("val2"), new_val:=Cell(), new_n:=Cell(), result:=Cell())

# Note: we need to only construct fib_iter's body if there is actually some value to do calculation on
# ("there being a value" == the construction thing has to be a propagator == compound propagator)

print(num_steps, val1, val2, new_val, new_n, result)
val1.write(0)
val2.write(1)
num_steps.write(6)
print(num_steps, val1, val2, new_val, new_n, result)  # We have recursion!


Cell('*NOTHING*') Cell('*NOTHING*')<val1> Cell('*NOTHING*')<val2> Cell('*NOTHING*') Cell('*NOTHING*') Cell('*NOTHING*')
Cell(6) Cell(0)<val1> Cell(1)<val2> Cell(1) Cell(5) Cell(13)


### Overloading

In [58]:
import inspect

def mult__0():
    return 1

def mult__1(x):
    return x

def mult__2(x, y):
    return x * y

def overload(*funcs, func_name=None, save_to_env=None):
    funcs = funcs or [v for k,v in globals().items() if k.startswith(f"{func_name}__")]
    arities = [len(inspect.signature(f).parameters) for f in funcs]
    if len(funcs) != len(arities):
        raise TypeError(f"All of the overloaded functions should have different numbers of parameters: {funcs}")
    dispatch = {a:f for a, f in zip(arities, funcs)}
    def inner(*args):
        func = dispatch[len(args)]
        return func(*args)
    if save_to_env is not None:
        if func_name is None:
            raise ValueError(f"If 'save_to_env' to store the overloaded function in is provided, we also need a 'func_name'")
        # print(f"Assigning '{func_name}' to env {save_to_env}")
        save_to_env[func_name] = inner
    return inner

def local_func():
    print("local_func:")
    mult = overload(mult__0, mult__1, mult__2)
    print(f"{mult()=}      (neutral element)")
    print(f"{mult(3)=}     (identity)")
    print(f"{mult(3, 4)=} (multiplication)")

local_func()
print(f"{locals().get('mult')=}")

def local_func2():
    print("local_func2:")
    overload(mult__0, mult__1, mult__2, func_name="mult", save_to_env=locals())
    print(f"{mult()=}      (neutral element)")
    print(f"{mult(3)=}     (identity)")
    print(f"{mult(3, 4)=} (multiplication)")

# local_func2()  # Strange that it can't find mult even though it's in locals

# Global
overload(mult__0, mult__1, mult__2, func_name="mult", save_to_env=globals())
print(f"Global env: {mult(2, 2)=}")
del(mult)

mult = overload(mult__0, mult__1, mult__2)
print(f"Global with assignment: {mult(2, 2)=}")
del(mult)

# Another way to use it (slower, but shorter to write):
def add(*args):
    def add0():
        return 0
    def add1(x):
        return x
    def add2(x, y):
        return x + y
    return overload(add0, add1, add2)(*args)

print("All-in-one definition:")
print(f"{add()=}")
print(f"{add(2)=}")
print(f"{add(3, 4)=}")

local_func:
mult()=1      (neutral element)
mult(3)=3     (identity)
mult(3, 4)=12 (multiplication)
locals().get('mult')=None
Global env: mult(2, 2)=4
Global with assignment: mult(2, 2)=4
All-in-one definition:
add()=0
add(2)=2
add(3, 4)=7


### Weird FizzBuzz

In [ ]:
def fizzbuzz_naive():
    for i in range(1, 101):
        s = ""
        if i % 3 == 0:
            s = "Fizz"
        if i % 5 == 0:
            s += "Buzz"
        print(s or i)

def fizzbuzz_short(n=1):
    print((n % 3 == 0) * "Fizz" + (n % 5 == 0) * "Buzz" or n)
    n == 100 or fizzbuzz(n + 1)

def fizzbuzz_geek():
    for i in range(1, 101):
        print([i, "Fizz", "Buzz", "FizzBuzz"][3 ^ (bool(i % 3) + 2 * bool(i % 5))])

def fizzbuzz_data_scientist(start=1, end=100):
    nums = range(start, end+1)
    conds = [3, 5]
    conds.append(conds[0] * conds[1])
    bool_list = [i / c == i // c for i in nums for c in conds]
    # print(bool_list, len(bool_list))
    for i in range(0, len(bool_list), len(conds)):
        tf_vals = bool_list[i : i + len(conds)]
        i = i // len(conds)
        result = i + 1
        if tf_vals[0] or tf_vals[1]:
            result = "Fizz" if tf_vals[0] else "Buzz"
        if tf_vals[2]:
            try:
                int(result)
            except ValueError:
                result = "FizzBuzz"
        print(result)
    print("Do I get the job?")

In [569]:
20 / 5

4.0

In [570]:
0 ^ 3

3

In [581]:
[1,2,3,4,5,6][1 : 2]

[2]

In [574]:
list(range(0, 7, 2))

[0, 2, 4, 6]